# Vai Como Pode 6

In [9]:
import os
import sys

In [3]:
run __init__.py

SyntaxError: invalid syntax (Temp/ipykernel_21068/1800883056.py, line 1)

In [4]:
bla()

NameError: name 'bla' is not defined

In [3]:
config.blem

[11]

In [4]:
class TP(object):
    def __init__(self,env):
        self.env = env
        self.name               = STARTING_NAME #name of the agent, is a string, normally something like TP_01
        self.genre              = 'TP' #genre, we do not use type, because type is a dedicated command of python, is also a string
        self.subgenre           = STARTING_TECHNOLOGY.get('source') #subgenre or source, we used to use subgenre a lot, now it's kind of a legacy. Is a number, 1 is wind, 2 is solar, 4 is biomass and 5 is hydrogen. 0 and 3 are not used because they are the fossil options
        self.wallet             = STARTING_WALLET #wallet or reserves, or savings, etc. How much the agent has? is a number
        self.profits            = 0 #profits of the agent, is a number
        self.capacity           = STARTING_CAPACITY #productive capacity of the agent, is a number used to determine the CAPEX of its technology
        self.RandD              = 0 #how much money was put into R&D? Is a number
        self.EorM               = STARTING_TECHNOLOGY.get('EorM') #does the agent use electricity or molecules? is a string (either 'E' or 'M') and is used a lot
        self.innovation_index   = 0 #index of innovation. Kind of a legacy, was used to analyze innovation
        self.Technology         = STARTING_TECHNOLOGY #the technology dict. Is a dictionary with the characteristics of the technology
        self.self_NPV           = {} #The NPV of a unit of investment. Is a dictionary: e.g. self_NPV={'value' : 2000, 'MWh' : 30}
        self.RnD_threshold      = STARTING_RnD_THRESHOLD #what is the threshold of R&D expenditure necessary to innovate? Is a number
        self.capacity_threshold = STARTING_CAPACITY_THRESHOLD #what is the threshold of investment in productive capacity in order to start decreasing CAPEX costs?
        self.dd_profits         = {0:0, 1:0, 2:0} if STARTING_TECHNOLOGY.get( 'EorM') == 'E' else {3:0, 4:0, 5:0} #same as profits, but as dict. Makes accounting faster and simpler
        self.dd_source          = STARTING_DD_SOURCE #This, my ganzirosis, used to be the Tactics. It is the first of the ranked dictionaries. It goes a little sumthing like dis: dd = {'current' : 2, 'ranks' : {0: 3500, 1: 720, 2: 8000}}. With that we have the current decision for the variable or thing and on the ranks we have the score for 
        self.decision_var       = STARTING_DECISION_VAR #this is the value of the decision variable. Is a number between -1 and 1
        self.action             = "keep" #this is the action variable. It can be either 'keep', 'change' or 'add' 
        self.dd_responsiveness  = STARTING_RESPONSIVENESS #this is the responsiveness, follows the current ranked dictionary
        self.dd_qual_vars       = STARTING_QUAL_VARS #this tells the agent the qualitative variables in a form {0 : 'name of the zeroth variable', 1 : 'name of the first variable', 2 : 'name of the second variable'}
        self.dd_backwardness    = STARTING_BACKWARDNESS #also a ranked dictionary, this one tells the backwardness of agents
        self.dd_avg_time        = STARTING_DD_AVG_TIME #also a ranked dictionary, this one tells the average time for deciding if change is necessary
        self.dd_discount           = STARTING_DISCOUNT #discount factor. Is a ranked dictionary
        self.cap_conditions     = STARTING_CAP_CONDITIONS #there are the cap conditions for this technology, being a dictionary following this example {'char' : 'CAPEX', 'cap' : 20000, 'increase' : 0.5}
        self.capped             = False #boolean variable to make the capping easier
        self.strategy           = STARTING_STRATEGY #the initial strategy of the agent, can be to reivnest into producetive capacity or R&D
        self.dd_strategies      = STARTING_STRATEGIES #initial strategy for the technology provider. Is a ranked dictionary
        
        self.action = env.process(self.run_TP(
            self.name              ,               
            self.genre             , 
            self.subgenre          , 
            self.wallet            , 
            self.profits           , 
            self.capacity          , 
            self.RandD             , 
            self.EorM              , 
            self.innovation_index  , 
            self.Technology        , 
            self.self_NPV          , 
            self.RnD_threshold     , 
            self.capacity_threshold,
            self.dd_profits        , 
            self.dd_source         , 
            self.decision_var      ,
            self.action            , 
            self.dd_responsiveness , 
            self.dd_qual_vars      , 
            self.dd_backwardness   , 
            self.dd_avg_time       , 
            self.dd_discount       ,
            self.cap_conditions    ,
            self.capped            ,
            self.dd_strategies     ))        
        
    def run_TP(self,
               name              ,
               genre             ,
               subgenre          ,
               wallet            ,
               profits           ,
               capacity          ,
               RandD             ,
               EorM              ,
               innovation_index  ,
               Technology        ,
               self_NPV          ,
               RnD_threshold     ,
               capacity_threshold,
               dd_profits        ,
               dd_source         ,
               decision_var      ,
               action            ,
               dd_responsiveness ,
               dd_qual_vars      ,
               dd_backwardness   ,
               dd_avg_time       ,
               dd_discount       ,
               cap_conditions    ,
               capped            ,
               dd_strategies     ):
        
        global CONTRACTS
        global MIX
        global AGENTS
        global AGENTS_r
        global TECHNOLOGIC
        global TECHNOLOGIC_r
        global r
        global TACTIC_DISCOUNT
        global AMMORT
        global rNd_INCREASE
        global RADICAL_THRESHOLD
        
        while True:
            
            #################################################################
            #                                                               #
            #     Before anything, we must the current values of each of    #
            #        the dictionaries that we use and other variables       #
            #                                                               #
            #################################################################
            
            list_of_strikeables = [dd_responsiveness, dd_qual_vars, dd_backwardness, dd_avg_time, dd_discount, dd_strategies]
            
            source         = dd_source.get(        'current')         
            responsiveness = dd_responsiveness.get('current') 
            qual_vars      = dd_qual_vars.get(     'current')      
            backwardness   = dd_backwardness.get(  'current')   
            avg_time       = dd_avg_time.get(      'current')
            discount       = dd_discount.get(      'current')
            strategy       = dd_strategies.get(    'current')
            
            value          = decision_var
            profits        = 0 #in order to get the profits of this period alone
            
            #################################################################
            #                                                               #
            #    First, the Technology provider closes any new deals and    #
            #                        collect profits                        #
            #                                                               #
            #################################################################

            if env.now > 0:
                for _ in CONTRACTS.get(env.now-1):
                    i = CONTRACTS.get(env.now-1).get(_)
                    if i.get('receiver') == name and i.get('status') == 'payment':
                        wallet    += i.get('value')
                        profits   += i.get('value')
                        """ we also have to update the sales_MWh entry, to indicate to the policy makers how much MWh of each source is there  """
                        j = dd_profits.get('ranks')
                        j.update({source : j.get(source)+ i.get('value')})
                        
            #################################################################
            #                                                               #
            #    Now, on to check if change is on and if there is a strike  #
            #                                                               #
            #################################################################
            
            if env.now > 0 and (action == 'add' or 'change'):
                striked = striking_FF(list_of_strikeables)
                
                for entry in range(0,len(list_of_strikeables)):
                    list_of_strikeables[entry] = striked[entry]
                action = 'keep' # we already changed, now back to business
                
            #################################################################
            #                                                               #
            #    Now, the Technology provider gets any incentive that is    #
            #                        available to it                        #
            #                                                               #
            #################################################################
            if env.now > 0:
                for _ in CONTRACTS.get(env.now-1):
                    i = CONTRACTS.get(env.now-1).get(_)
                    if i.get('receiver') == name and i.get('sender') == 'TPM':
                        wallet += i.get('value')
                        
            #################################################################
            #                                                               #
            #         The TP then has to 1) adjust the base capex to        #
            #         the productive capacity (if the technology is         #
            #  non-transportable; 2) change the strategy if the action was  #
            #  to change it; 3) spend the available money into imitation,   #
            # innovation or productive capacity; 4) check if the TP reached #
            #     the threshold of innovation/imitation and change the      #
            #    Technology dictionary if it reached that; 5) update the    #
            #    global TECHNOLOGY dictionary; and lastly 6) do the self    #
            #           NPV entry for the AGENTS dictionary later.          #
            #           8) Oh, it must also the capping process             #
            #                                                               #
            #################################################################
            
            """ 1)  we have to get the base-CAPEX and adjust it to the productive capacity of the TP (only if its technology is not transportable) """
            if Technology.get('transport') == False:
                """ if the technology is not transportable, then the productive capacity impacts on the CAPEX """
                if env.now == 0:
                    """ if we are in the first period, then we have to get the starting CAPEX and tell the TP that this is his base capex, because there is no base_capex already """
                    i = Technology.get('CAPEX')
                    Technology.update({"base_CAPEX" : i})
                j = Technology.get('base_CAPEX')
                """ we have to produce the actual CAPEX, with is the base_CAPEX multiplied by euler's number to the power of the ratio of how many times the base capex is greater than the capacity itself multiplied by the threshold of capacity"""
                new_capex = min(j, (capacity_threshold/capacity)*j)
                Technology.update({
                    'CAPEX' : j* new_capex
                })
            else:
                """ the technology is transportable (e.g. solar panels)"""
                i = Technology.get('CAPEX')
                Technology.update({"base_CAPEX" : i})
                
            """3) now, if the TP has money, it will spend it on either capacity, imitation or innovation"""
            if wallet > 0 and value>0:
                wallet -= wallet * value
                """ having money, the TP will spend a portion (given by the value) of its wallet on something """
                if strategy == 'innovation':
                    """ if the strategy is to do R&D, then it will do R&D"""
                    RandD += wallet * value
                    innovation_index += wallet * value
                    #print(name, 'got', wallet * value, 'more innovation to its roster, and now innovation is', innovation_index)
                else:
                    """ if not, then it will spend on productive capacity"""
                    capacity += wallet * value
                    #print(name, 'got', wallet * value, 'more capacity to its roster, and now capacity is', capacity)
            
            """4) we have to check if the TP reached the threshold for innovation or imitation"""
            if RandD > RnD_threshold and (strategy == 'innovation' or strategy == 'imitation'):
                """ if we reached the threshold, then we to set the bar of the RnD """
                RnD_threshold += RandD
                """ then we get the 'a' which can either be a poisson + normal for innovation, or a simple binomial. values above or equal (for the imitation) 1 indicate that innovation or imitation occured """
                a = np.random.poisson(1) + np.random.normal(0,1)
                
                if a >= 1:
                    """ we are dealing with innovation """
                    RnD_threshold *= rNd_INCREASE * a
                    """ we have to check where did the innovation occur"""
                    what_on = random.choice(['base_CAPEX','OPEX','MW'])
                    """ if innovation ocurred then we multiply it"""
                    if what_on == 'MW':
                        new_what_on = a * Technology.get(what_on)
                    else:
                        new_what_on = (1 / a) * Technology.get(what_on)
                    Technology.update({ what_on : new_what_on })
                    
                    if a > RADICAL_THRESHOLD:
                        """ if we reached over the radical innovation threshold we have to signal it """
                        radical_or_not = 'last_radical_innovation'
                    else:
                        """ we did not reach over that threshold, so it was a marginal innovation """
                        radical_or_not = 'last_marginal_innovation'
                    Technology.update({
                        radical_or_not : env.now
                    })
            
            """6) lastly, we have to get the self.NPV of its current technology"""
            if env.now>0:
                i = TECHNOLOGIC.get(env.now-1).get(name)
                price = weighting_FF(env.now-1, 'price', 'MWh', MIX, EorM=EorM)
                interest_r = weighting_FF(env.now-1, 'interest_r', 'MWh', MIX, EorM=EorM, discount=discount)
                self_NPV.update({'value' : npv_generating_FF(r, i.get('lifetime'), 1, i.get('MW'), i.get('building_time'), i.get('CAPEX'), i.get('OPEX'), price.get(i.get('source')), i.get('CF'), AMMORT),
                                 'MWh' : i.get('MW')
                })
                
            """ 8) we must also check if the capping process is on"""
            
            if len(MIX.get(env.now-1)):
                #if there is no capping, we must first make sure that it has not started
                now = finding_FF(MIX.get(env.now-1), 'MW', 'sum', {'EP' : name})
                if now > cap_conditions['cap']:
                    capped == True
                
                else:
                    capped == False
            
            if capped == True:
                #capping process is on, so we have to make sure that the capacity increased
                previous = finding_FF(MIX.get(env.now-2), 'MW', 'sum', {'EP' : name})
                
                if now > previous:
                    #the capacity increased, so we have to apply the capping conditions
                    Technology.update({
                        cap_conditions['char'] : Technology.get( cap_conditions['char'])*(1+cap_conditions['increase'])
                    })
                
            """5) we also have to update the TECHNOLOGIC dictionary for the next period with the current technology (applying any changes if there were any whatsover)"""
            TECHNOLOGIC.get(env.now).update({name : Technology})
                
            #################################################################
            #                                                               #
            #  And finally, the Technology provider decides what to do in   #
            #                        the next period                        #
            #                                                               #
            #################################################################
            
            """ TPs don't compare sources, so there is no reporting_FF here"""
            if env.now>0:
                
                decision_var = private_deciding_FF(name)
                
                decisions = evaluating_FF(name)
                
                action = decisions['action']
                
                
            #################################################################
            #                                                               #
            #    Before leaving, the agent must uptade the outside world    #
            #                                                               #
            #################################################################

            AGENTS.get(env.now).update({name : {
                "name"               : name              ,
                "genre"              : genre             ,
                "subgenre"           : subgenre          ,
                "wallet"             : wallet            ,
                "profits"            : profits           ,
                "capacity"           : capacity          ,
                "RandD"              : RandD             ,
                "EorM"               : EorM              ,
                "innovation_index"   : innovation_index  ,
                "Technology"         : Technology        ,
                "self_NPV"           : self_NPV          ,
                "RnD_threshold"      : RnD_threshold     ,
                "capacity_threshold" : capacity_threshold, 
                "dd_profits"         : dd_profits        ,
                "dd_source"          : dd_source         ,
                "decision_var"       : decision_var      ,
                "action"             : action            ,
                "dd_responsiveness"  : dd_responsiveness ,
                "dd_qual_vars"       : dd_qual_vars      ,
                "dd_backwardness"    : dd_backwardness   ,
                "dd_avg_time"        : dd_avg_time       ,
                "dd_discount"        : dd_discount       ,
                "cap_conditions"     : cap_conditions    ,
                "capped"             : capped            ,
            }})
            
            profits_dedicting_FF(name)
            post_evaluating_FF(decisions['strikes'], name)
            
            yield env.timeout(1)

In [29]:
class TPM(object):
    def __init__(self,env):
        self.env               = env
        self.genre             = 'TPM'
        self.subgenre          = 'TPM'
        self.name              = 'TPM'
        self.wallet            = STARTING_WALLET
        self.dd_policy         = STARTING_DD_POLICY
        self.dd_source         = STARTING_DD_SOURCE
        self.decision_var      = STARTING_DECISION_VAR
        self.disclosed_var     = STARTING_DECISION_VAR
        self.action            = 'keep'
        self.dd_responsiveness = STARTING_DD_RESPONSIVENESS
        self.dd_qual_vars      = STARTING_DD_QUAL_VARS
        self.dd_backwardness   = STARTING_DD_BACKWARDNESS
        self.dd_avg_time       = STARTING_DD_AVG_TIME
        self.dd_discount       = STARTING_DD_DISCOUNT
        self.dd_policy         = STARTING_DD_POLICY
        self.policies          = STARTING_POLICIES
        self.dd_index          = STARTING_DD_INDEX
        self.index_per_source  = {1:0,2:0,4:0,5:0}
        self.dd_eta            = STARTING_DD_ETA
        self.dd_ambition       = STARTING_DD_AMBITION
        self.dd_target         = STARTING_DD_TARGET
        self.dd_rationale      = STARTING_DD_RATIONALE
        
        
        self.action=env.process(self.run_TPM(self.genre             , 
                                             self.subgenre          , 
                                             self.name              , 
                                             self.wallet            , 
                                             self.dd_policy         , 
                                             self.dd_source         , 
                                             self.decision_var      , 
                                             self.disclosed_var     , 
                                             self.action            , 
                                             self.dd_responsiveness , 
                                             self.dd_qual_vars      , 
                                             self.dd_backwardness   , 
                                             self.dd_avg_time       , 
                                             self.dd_discount       , 
                                             self.dd_policy         , 
                                             self.policies          , 
                                             self.dd_index          , 
                                             self.index_per_source  , 
                                             self.dd_eta            , 
                                             self.dd_ambition       , 
                                             self.dd_target         , 
                                             self.dd_rationale      ))
        
    def run_TPM(self         ,
                genre             ,
                subgenre          ,
                name              ,
                wallet            ,
                dd_policy         ,
                dd_source         ,
                decision_var      ,
                disclosed_var     ,
                action            ,
                dd_responsiveness ,
                dd_qual_vars      ,
                dd_backwardness   ,
                dd_avg_time       ,
                dd_discount       ,
                policies          ,
                dd_index          ,
                index_per_source  ,
                dd_eta            ,
                dd_ambition       ,
                dd_target         ,
                dd_rationale      ):
        
        global CONTRACTS
        global MIX
        global AGENTS
        global TECHNOLOGIC
        global r     
        global POLICY_EXPIRATION_DATE
        global AMMORT
        global TACTIC_DISCOUNT
        global INSTRUMENT_TO_SOURCE_DICT
        
        while True:
            
            #################################################################
            #                                                               #
            #              First, the TPM checks if it got an               #
            #            strike, adds or changes its main policy            #
            #                                                               #
            #################################################################
            
            list_of_strikeables = [dd_policy, dd_source, dd_responsiveness, dd_qual_vars, dd_backwardness, dd_avg_time, dd_discount, dd_policy, dd_index, dd_eta, dd_ambition, dd_target, dd_rationale]
            
            policy         =         dd_policy['current']
            source         =         dd_source['current']
            responsiveness = dd_responsiveness['current']
            backwardness   =   dd_backwardness['current']
            avg_time       =       dd_avg_time['current']
            discount       =       dd_discount['current']
            index          =           indexing_FF('TPM') if env.now>0 else dd_index['current']
            eta_acc        =            dd_eta['current']
            ambition       =       dd_ambition['current']
            rationale      =      dd_rationale['current']
            value          =                disclosed_var
            
            if env.now > 0 and (action == 'add' or 'change'):
                striked = striking_FF(list_of_strikeables) #with this we have a different list of strikeables
                
                for entry in range(0,len(list_of_strikeables)):
                    
                    if list_of_strikeables[entry]['current'] == striked[entry]:
                        #that dictionary was not the changed one, so we can just update it
                        list_of_strikeables[entry] = striked[entry]
                        
                    else:
                        #alright, that was the one that changed
                        
                        Policies = policymaking_FF(dd, Policies, disclosed_var) if action == 'change' else policymaking_FF(dd, Policies, disclosed_var , add=True)
                
                action = 'keep' # we already changed, now back to business
            
            #################################################################
            #                                                               #
            #             Now, the TPM gives out the incentives             #
            #                                                               #
            #################################################################
            
            policy_pool = []
            policy_pool.append(policy)
            policy_pool.append(policies) #with this we a temporary list with first the current policy and afterwars all the other policies
                
            if env.now >=2 :
                for entry in policy_pool:
                    instrument = entry.get('instrument')
                    source     = entry.get('source')
                    budget     = entry.get('budget') if 'budget' in entry else value * wallet
                    value      = disclosed_var if 'value' not in entry else entry.get('value')

                if instrument == 'supply':  

                    firms = []
                    for _ in AGENTS.get(env.now-1):
                        i = AGENTS.get(env.now-1).get(_)
                        if i.get('genre') == 'TP' and (i.get('source') in INSTRUMENT_TO_SOURCE_DICT.get( source)):
                            firms.append(_)
                    
                    if len(firms) > 0:
                        """ we have to be certain that there are companies to be inbcentivised and now divides the possible incentive by the number of companies """
                        #print('incentivised_firms', incentivised_firms)
                        incentive = budget / len(firms)

                        """ and now we give out the incentives"""   
                        for TP in incentivised_firms:
                            code=uuid.uuid4().int
                            CONTRACTS.get(env.now).update({
                                code : {
                                    'sender'   : name     ,
                                    'receiver' : TP       ,
                                    'status'   : 'payment',
                                    'value'    : incentive}})
                else:
                    """
                    demmand-side incentives
                    """
                    print('TBD')

                """ and now back to the actual variables for the current policy"""
                instrument  = Policies[0]['instrument']
                source      = Policies[0]['source']    
                value       = disclosed_var
            
            #################################################################
            #                                                               #
            #    Now, the TPM analyses what happened to the system due to   #
            #                       its intervention                        #
            #                                                               #
            #################################################################

            add_source = source_reporting_FF(name)
            
            for entry in dd_source['ranks']:
                dd_source['ranks'][entry] *= (1-discount)
                dd_source['ranks'][entry] += add_source[entry]
            
            #################################################################
            #                                                               #
            #         And then, the TPM will decide what to do next         #
            #                                                               #
            #################################################################

            if env.now>0:
                
                decision_var = max(0,min(1, public_deciding_FF(name)))
                disclosed_var = thresholding_FF(responsivity, disclosed_var, decision_var)
            
            #################################################################
            #                                                               #
            #    Before leaving, the agent must uptade the outside world    #
            #                                                               #
            #################################################################

            AGENTS.get(env.now).update({
                name : {
                    "genre"             : genre             ,
                    "subgenre"          : subgenre          ,
                    "name"              : name              ,
                    "wallet"            : wallet            ,
                    "dd_policy"         : dd_policy         ,
                    "dd_source"         : dd_source         ,
                    "decision_var"      : decision_var      ,
                    "disclosed_var"     : disclosed_var     ,
                    "action"            : action            ,
                    "dd_responsiveness" : dd_responsiveness ,
                    "dd_qual_vars"      : dd_qual_vars      ,
                    "dd_backwardness"   : dd_backwardness   ,
                    "dd_avg_time"       : dd_avg_time       ,
                    "dd_discount"       : dd_discount       ,
                    "dd_policy"         : dd_policy         ,
                    "policies"          : policies          ,
                    "dd_index"          : dd_index          ,
                    "index_per_source"  : index_per_source  ,
                    "dd_eta"            : dd_eta            ,
                    "dd_ambition"       : dd_ambition       ,
                    "dd_target"         : dd_target         ,
                    "dd_rationale"      : dd_rationale      ,
                    "decision_var"      : decision_var      ,
                    "disclosed_var"     : disclosed_var     ,
                    "policy"            : policy            ,
                    "source"            : source            ,
                    "responsiveness"    : responsiveness    ,
                    "backwardness"      : backwardness      ,
                    "avg_time"          : avg_time          ,
                    "discount"          : discount          ,
                    "index"             : index             ,
                    "eta_acc"           : eta_acc           ,
                    "ambition"          : ambition          ,
                    "rationale"         : rationale         ,
                    "value"             : value             ,
                }})
            
            post_evaluating_FF(decisions['strikes'], name)

            yield env.timeout(1)

In [32]:
class EPM(object):
    def __init__(self,env):
        self.env               = env
        self.genre             = 'EPM'
        self.subgenre          = 'EPM'
        self.name              = 'EPM'
        self.wallet            = STARTING_WALLET
        self.PPA_expiration    = STARTING_PPA_LIMIT
        self.PPA_limit         = STARTING_PPA_LIMIT
        self.auction_countdown = 0 
        self.auction_time      = False
        self.COUNTDOWN         = STARTING_COUNTDOWN
        self.dd_policy         = STARTING_DD_POLICY
        self.dd_source         = STARTING_DD_SOURCE
        self.decision_var      = STARTING_DECISION_VAR
        self.disclosed_var     = STARTING_DECISION_VAR
        self.action            = 'keep'
        self.dd_responsiveness = STARTING_DD_RESPONSIVENESS
        self.dd_qual_vars      = STARTING_DD_QUAL_VARS
        self.dd_backwardness   = STARTING_DD_BACKWARDNESS
        self.dd_avg_time       = STARTING_DD_AVG_TIME
        self.dd_discount       = STARTING_DD_DISCOUNT
        self.dd_policy         = STARTING_DD_POLICY
        self.policies          = STARTING_POLICIES
        self.dd_index          = STARTING_DD_INDEX
        self.index_per_source  = {1:0,2:0,4:0,5:0}
        self.dd_eta            = STARTING_DD_ETA
        self.dd_ambition       = STARTING_DD_AMBITION
        self.dd_target         = STARTING_DD_TARGET
        self.dd_rationale      = STARTING_DD_RATIONALE
        self.auction_capacity  = STARTING_AUCTION_CAPACITY
        
        self.action=env.process(self.run_EPM(self.genre             ,
                                             self.subgenre          ,
                                             self.name              ,
                                             self.wallet            ,
                                             self.PPA_expiration    ,
                                             self.PPA_limit         ,
                                             self.auction_countdown ,
                                             self.auction_time      ,
                                             self.COUNTDOWN         ,
                                             self.dd_policy         ,
                                             self.dd_source         ,
                                             self.decision_var      ,
                                             self.disclosed_var     ,
                                             self.action            ,
                                             self.dd_responsiveness ,
                                             self.dd_qual_vars      ,
                                             self.dd_backwardness   ,
                                             self.dd_avg_time       ,
                                             self.dd_discount       ,
                                             self.dd_policy         ,
                                             self.dd_index          ,
                                             self.index_per_source  ,
                                             self.dd_eta            ,
                                             self.dd_ambition       ,
                                             self.dd_target         ,
                                             self.dd_rationale      ,
                                             self.auction_capacity  ))
        
    def run_EPM(self,
                genre             ,
                subgenre          ,
                name              ,
                wallet            ,
                PPA_expiration    ,
                PPA_limit         ,
                auction_countdown ,
                auction_time      ,
                COUNTDOWN         ,
                dd_policy         ,
                dd_source         ,
                decision_var      ,
                disclosed_var     ,
                action            ,
                dd_responsiveness ,
                dd_qual_vars      ,
                dd_backwardness   ,
                dd_avg_time       ,
                dd_discount       ,
                policies          ,
                dd_index          ,
                index_per_source  ,
                dd_eta            ,
                dd_ambition       ,
                dd_target         ,
                dd_rationale      ,
                auction_capacity  ):
        
        global CONTRACTS
        global MIX
        global AGENTS
        global TECHNOLOGIC
        global DEMAND
        global M_CONTRACT_LIMIT
        global AUCTION_WANTED_SOURCES
        global AMMORT
        global TACTIC_DISCOUNT
        global INSTRUMENT_TO_SOURCE_DICT
        global MARGIN
        
        while True:
            
            #################################################################
            #                                                               #
            #              First, the TPM checks if it got an               #
            #            strike, adds or changes its main policy            #
            #                                                               #
            #################################################################
            
            list_of_strikeables = [dd_policy, dd_source, dd_responsiveness, dd_qual_vars, dd_backwardness, dd_avg_time, dd_discount, dd_policy, dd_index, dd_eta, dd_ambition, dd_target, dd_rationale]
            
            policy         =         dd_policy['current']
            source         =         dd_source['current']
            responsiveness = dd_responsiveness['current']
            backwardness   =   dd_backwardness['current']
            avg_time       =       dd_avg_time['current']
            discount       =       dd_discount['current']
            index          =           indexing_FF('EPM') if env.now>0 else dd_index['current']
            eta_acc        =            dd_eta['current']
            ambition       =       dd_ambition['current']
            rationale      =      dd_rationale['current']
            value          =                disclosed_var
            
            if env.now > 0 and (action == 'add' or 'change'):
                striked = striking_FF(list_of_strikeables) #with this we have a different list of strikeables
                
                for entry in range(0,len(list_of_strikeables)):
                    
                    if list_of_strikeables[entry]['current'] == striked[entry]:
                        #that dictionary was not the changed one, so we can just update it
                        list_of_strikeables[entry] = striked[entry]
                        
                    else:
                        #alright, that was the one that changed
                        
                        Policies = policymaking_FF(dd, Policies, disclosed_var) if action == 'change' else policymaking_FF(dd, Policies, disclosed_var , add=True)
                
                action = 'keep' # we already changed, now back to business
            
            #################################################################
            #                                                               #
            #       Before doing policy, the policy maker must decide       #
            #                       which plants enter                      #
            #                                                               #
            #################################################################
            
            """ since the policy makers act after private agents, they are looking at the env.now, not the env.now-1 """
            if env.now > 0 and len(MIX.get(env.now)) > 0:
                """ 
                First, we contract and precify the electricity projects
                """
                
                possible_projects = [] 
                for _ in MIX.get(env.now):
                    """ we build the list of possible projects, i.e., projects deployed or already built"""
                    j = MIX.get(env.now).get(_)
                    if ((j.get('status') == 'built' or j.get('status') == 'contracted') and j.get('EorM') == 'E'):
                        possible_projects.append(j)
                """ now, we sort the list of dictionaries in terms of dispatchability and then OPEX (respecting the merit order)"""
                possible_projects = sorted(possible_projects, key=lambda x : (x.get('dispatchable'), x.get('OPEX')))
                chosen = []
                demand = DEMAND.copy().get(env.now).get('E')
                for plant in possible_projects:
                    if demand < 0:
                        """ if there is no more demand to be supplied, then the plant is not contracted"""
                        MIX.get(env.now).get(plant.get('code')).update({
                            'status' : 'built'
                        })
                    else:
                        """ if there is still demand to to be supplied, then, the power plant is contracted"""
                        MIX.get(env.now).get(plant.get('code')).update({
                            'status' : 'contracted'
                        })
                        chosen.append(plant)
                        demand -= plant.get('MWh')
                """ following the merit order, the system is precified in relation to its most costly unit"""
                chosen = sorted(chosen, key= lambda x : x.get('OPEX'))[-1]
                price = (chosen.get('OPEX') + (chosen.get('CAPEX') / chosen.get('lifetime'))) * (1 + MARGIN) / chosen.get('MWh')
                for i in possible_projects:
                    """ if the plant is auction contracted, then we do not mess with its price"""
                    if i.get('auction_contracted') != True: 
                        MIX.get(env.now).get(i.get('code')).update({
                            'price' : price})
                """ 
                now, we contract molecule projects
                """
                
                possible_projects=[] 
                demand=DEMAND.copy().get(env.now).get('M')
                """ since molecule projects are take of pay, if they are not contracted, then demand make contract them"""
                for _ in MIX.get(env.now):
                    j=MIX.get(env.now).get(_)
                    if j.get('status')  != 'contracted' and j.get('EorM') == 'M' and j.get('M_limit') != env.now:
                        possible_projects.append(j)
                    elif j.get('status')  == 'contracted' and j.get('EorM') == 'M' and j.get('M_limit') == env.now:
                        j.update({'status' : 'uncontracted'})
                        demand -= j.get('MWh')
                        
                """ this time, we sort by price, as the prices are defined by the agents """
                possible_projects = sorted(possible_projects, key = lambda x : (x.get('price')))
                for plant in possible_projects:
                    if demand < 0:
                        MIX.get(env.now).get(plant.get('code')).update({
                            'status' : 'uncontracted'
                        })
                    else:
                        MIX.get(env.now).get(plant.get('code')).update({
                            'status' : 'contracted',
                            'M_limit' : env.now + 1 + M_CONTRACT_LIMIT
                        })
                    demand -= plant.get('MWh')
                    
            #################################################################
            #                                                               #
            #           First, the EPM adds or changes its policy           #
            #                                                               #
            ################################################################# 
            
            policy_pool = []
            policy_pool.append(policy)
            policy_pool.append(policies) #with this we a temporary list with first the current policy and afterwars all the other policies
                
            if env.now >=2 :
                for entry in policy_pool:
                    instrument = entry.get('instrument')
                    source     = entry.get('source')
                    budget     = entry.get('budget') if 'budget' in entry else value * wallet
                    value      = disclosed_var if 'value' not in entry else entry.get('value')
                    
                    if instrument == 'carbon_tax':
                        """
                        CARBON-TAX
                        """    

                        for _ in TECHNOLOGIC.get(0):
                            i = TECHNOLOGIC.get(0).get(_)
                            j = TECHNOLOGIC.get(env.now + 1).get(_)
                            """ for the carbon-tax, if the source is thermal (0) or natural gas (3), then, its OPEX gets higher"""
                            if i.get('source') in [0,3]:
                                j.update({
                                    'OPEX' : i.get('OPEX') * (1 + value)
                                })
                                """ we can update in relation to the initial value, because there is no innovation for non-renewables"""

                    elif instrument == 'FiT':
                        """
                        Feed-in Tariff
                        """

                        """ we are actually using Feed-in premium, since we are adding a payment to the market price"""
                        for _ in MIX.get(env.now-1):
                            i=MIX.get(env.now-1).get(_)
                            if i.get('source') in INSTRUMENT_TO_SOURCE_DICT( target ) and i.get('auction_contracted') != True:
                                i.update({
                                    'price' : i.get('price') * (1 + value)
                                })
                                
                    else:
                        """
                        Auctions: these are a little bit more complicated
                        """

                        """ we do things with auction_countdown, which is a countdown for the auction period, which allows the EPM to collect projhects, and with auction_time, a boolean variable which signals if there is an auction underway"""
                        if auction_countdown > 0 and auction_time == True:
                            """ if the coundtdown has yet to reach zero but an Auction is underway, then we cut down one period from the auction_countdown """
                            auction_countdown -= 1
                        elif auction_time == False:
                            """ if we are doing an auction, but the auction_time is not signalling that an auction is underway, then we have to set it to true and trigger the countdown to the auction itself"""
                            auction_countdown = COUNTDOWN
                            auction_time = True
                            for j in range(env.now, env.now + COUNTDOWN):
                                """ as we starting the auction, we need to signal energy producers that their next projects of the target-source will be put inside the auction, for that we set up bids for each energy producer for each period from now until the deadline. This process creates bids for new agents also"""
                                for source in INSTRUMENT_TO_SOURCE_DICT.get( target ):
                                    AUCTION_WANTED_SOURCES.append( INSTRUMENT_TO_SOURCE_DICT.get( target ) )
                        elif auction_countdown == 0 and auction_time == True:
                            """ under these circumnstances, it is time to do the auction and contract projects for the ppas"""
                            auction_time = False
                            possible_projects = []
                            contracted_projects = []
                            for source in INSTRUMENT_TO_SOURCE_DICT.get( target ):
                                AUCTION_WANTED_SOURCES.remove( INSTRUMENT_TO_SOURCE_DICT.get( target ) )
                            """first we get the projects that were bidded"""
                            for i in range(env.now-COUNTDOWN, env.now-1): 
                                for _ in CONTRACTS.get(i):
                                    __ = CONTRACTS.get(i).get(_)
                                    j = __.copy()
                                    if j.get('bidded') == True:
                                        """ here we are adding the whole contract"""
                                        possible_projects.append(j)
                            """ we have to sort in terms of 'OPEX' """
                            possible_projects = sorted(possible_projects, key = lambda x :  x.get('OPEX'))
                            """ the capacity for auction is the value (in GW) multplied by 1000 (to become MW)"""
                            remaining_capacity = auction_capacity * value * 1000
                            for i in possible_projects:
                                if remaining_capacity > 0:
                                    """ if there is still capacity to contract, then the project is contracted"""
                                    contracted_projects.append(i.get('code'))
                                remaining_capacity -= i.get('capacity')
                            """ then we cycle throught the codes of the contracted projects in order to tell the proponent agents that their projects were contracted"""
                            for code in contracted_projects:
                                """ first we update it to include attributes exclusive to PPAs, such as the auction price, the boolean auction_contracted and the date of expiration of the price"""
                                CONTRACTS.get(env.now).update({code : {
                                    'status' : 'project',
                                    'price' : (possible_projects.get( i ).get('OPEX') + (possible_projects.get( i ).get('CAPEX') / possible_projects.get( i ).get('lifetime'))) * (1 + MARGIN) / possible_projects.get( i ).get('MWh'),
                                    'auction_contracted' : True,
                                    'price_expiration' : env.now + PPA_expiration + PPA_limit + 1,
                                    'limit' : env.now + PPA_limit + 1
                                }})

                            """ lastly, we have to get rid of the contracted projects. This is relevant specially when  there are multiple auctions happening"""
                            for i in range(env.now-1-COUNTDOWN, env.now-1): 
                                for _ in CONTRACTS.get(i):
                                    j=CONTRACTS.get(i).get(_)
                                    if j.get('code') in contracted_projects:
                                        j.update('bidded' == False)

                """ and now back to the actual variables for the current policy"""
                
                instrument  = Policies[0]['instrument']
                source      = Policies[0]['source']  
                value       = disclosed_var
            
            #################################################################
            #                                                               #
            #    Now, the EPM analyses what happened to the system due to   #
            #                       its intervention                        #
            #                                                               #
            #################################################################

            add_source = source_reporting_FF(name)
            
            for entry in dd_source['ranks']:
                dd_source['ranks'][entry] *= (1-discount)
                dd_source['ranks'][entry] += add_source[entry]
            
            #################################################################
            #                                                               #
            #         And then, the EPM will decide what to do next         #
            #                                                               #
            #################################################################

            if env.now>0:
                
                decision_var = max(0,min(1, public_deciding_FF(name)))
                disclosed_var = thresholding_FF(responsivity, disclosed_var, decision_var)
                
            #################################################################
            #                                                               #
            #    Before leaving, the agent must uptade the outside world    #
            #                                                               #
            #################################################################

            AGENTS.get(env.now).update({name : {
                "genre"             : genre             ,
                "subgenre"          : subgenre          ,
                "name"              : name              ,
                "wallet"            : wallet            ,
                "PPA_expiration"    : PPA_expiration    ,
                "PPA_limit"         : PPA_limit         ,
                "auction_countdown" : auction_countdown ,
                "auction_time"      : auction_time      ,
                "COUNTDOWN"         : COUNTDOWN         ,
                "dd_policy"         : dd_policy         ,
                "dd_source"         : dd_source         ,
                "decision_var"      : decision_var      ,
                "disclosed_var"     : disclosed_var     ,
                "action"            : action            ,
                "dd_responsiveness" : dd_responsiveness ,
                "dd_qual_vars"      : dd_qual_vars      ,
                "dd_backwardness"   : dd_backwardness   ,
                "dd_avg_time"       : dd_avg_time       ,
                "dd_discount"       : dd_discount       ,
                "dd_policy"         : dd_policy         ,
                "policies"          : policies          ,
                "dd_index"          : dd_index          ,
                "index_per_source"  : index_per_source  ,
                "dd_eta"            : dd_eta            ,
                "dd_ambition"       : dd_ambition       ,
                "dd_target"         : dd_target         ,
                "dd_rationale"      : dd_rationale      ,
                "auction_capacity"  : auction_capacity  ,
                "instrument"        : instrument        ,
                "source"            : source            ,
            }})
            
            post_evaluating_FF(decisions['strikes'], name)

            yield env.timeout(1)

In [33]:
class DBB(object):
    def __init__(self,env):
        self.env = env
        self.NPV_THRESHOLD_DBB    = STARTING_NPV_THRESHOLD_DBB
        self.guaranteed_contracts = []
        self.genre                = 'DBB'
        self.subgenre             = 'DBB'
        self.name                 = 'DBB'
        self.wallet               = STARTING_WALLET
        self.dd_policy            = STARTING_DD_POLICY
        self.dd_source            = STARTING_DD_SOURCE
        self.decision_var         = STARTING_DECISION_VAR
        self.disclosed_var        = STARTING_DECISION_VAR
        self.action               = 'keep'
        self.dd_responsiveness    = STARTING_DD_RESPONSIVENESS
        self.dd_qual_vars         = STARTING_DD_QUAL_VARS
        self.dd_backwardness      = STARTING_DD_BACKWARDNESS
        self.dd_avg_time          = STARTING_DD_AVG_TIME
        self.dd_discount          = STARTING_DD_DISCOUNT
        self.dd_policy            = STARTING_DD_POLICY
        self.policies             = STARTING_POLICIES
        self.dd_index             = STARTING_DD_INDEX
        self.index_per_source     = {1:0,2:0,4:0,5:0}
        self.dd_eta               = STARTING_DD_ETA
        self.dd_ambition          = STARTING_DD_AMBITION
        self.dd_target            = STARTING_DD_TARGET
        self.dd_rationale         = STARTING_DD_RATIONALE
        self.financing_index      = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
        self.Portfolio            = STARTING_PORTFOLIO
        self.receivable           = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
        self.accepted_sources     = ACCEPTED_SOURCES
        self.car_ratio            = 0
        
        self.action=env.process(self.run_DBB(self.NPV_THRESHOLD_DBB    , 
                                             self.guaranteed_contracts , 
                                             self.genre                , 
                                             self.subgenre             , 
                                             self.name                 , 
                                             self.wallet               , 
                                             self.dd_policy            , 
                                             self.dd_source            , 
                                             self.decision_var         , 
                                             self.disclosed_var        , 
                                             self.action               , 
                                             self.dd_responsiveness    , 
                                             self.dd_qual_vars         , 
                                             self.dd_backwardness      , 
                                             self.dd_avg_time          , 
                                             self.dd_discount          , 
                                             self.dd_policy            , 
                                             self.policies             , 
                                             self.dd_index             , 
                                             self.index_per_source     , 
                                             self.dd_eta               , 
                                             self.dd_ambition          , 
                                             self.dd_target            , 
                                             self.dd_rationale         , 
                                             self.financing_index      , 
                                             self.Portfolio            , 
                                             self.receivable           , 
                                             self.accepted_sources     , 
                                             self.car_ratio             ))
        
    def run_DBB(self, 
                NPV_THRESHOLD_DBB    ,
                guaranteed_contracts ,
                genre                ,
                subgenre             ,
                name                 ,
                wallet               ,
                dd_policy            ,
                dd_source            ,
                decision_var         ,
                disclosed_var        ,
                action               ,
                dd_responsiveness    ,
                dd_qual_vars         ,
                dd_backwardness      ,
                dd_avg_time          ,
                dd_discount          ,
                dd_policy            ,
                policies             ,
                dd_index             ,
                index_per_source     ,
                dd_eta               ,
                dd_ambition          ,
                dd_target            ,
                dd_rationale         ,
                financing_index      ,
                Portfolio            ,
                receivable           ,
                accepted_sources     ,
                car_ratio             ):
        
        global CONTRACTS
        global MIX
        global AGENTS
        global TECHNOLOGIC
        global r     
        global POLICY_EXPIRATION_DATE
        global INSTRUMENT_TO_SOURCE_DICT
        global RISKS
        
        while True:
            
            #################################################################
            #                                                               #
            #              First, the DBB checks if it got an               #
            #            strike, adds or changes its main policy            #
            #                                                               #
            #################################################################
            
            list_of_strikeables = [dd_policy, dd_source, dd_responsiveness, dd_qual_vars, dd_backwardness, dd_avg_time, dd_discount, dd_policy, dd_index, dd_eta, dd_ambition, dd_target, dd_rationale]
            
            policy         =         dd_policy['current']
            source         =         dd_source['current']
            responsiveness = dd_responsiveness['current']
            backwardness   =   dd_backwardness['current']
            avg_time       =       dd_avg_time['current']
            discount       =       dd_discount['current']
            index          =           indexing_FF('EPM') if env.now>0 else dd_index['current']
            eta_acc        =            dd_eta['current']
            ambition       =       dd_ambition['current']
            rationale      =      dd_rationale['current']
            value          =                disclosed_var
            
            if env.now > 0 and (action == 'add' or 'change'):
                striked = striking_FF(list_of_strikeables) #with this we have a different list of strikeables
                
                for entry in range(0,len(list_of_strikeables)):
                    
                    if list_of_strikeables[entry]['current'] == striked[entry]:
                        #that dictionary was not the changed one, so we can just update it
                        list_of_strikeables[entry] = striked[entry]
                        
                    else:
                        #alright, that was the one that changed
                        
                        Policies = policymaking_FF(dd, Policies, disclosed_var) if action == 'change' else policymaking_FF(dd, Policies, disclosed_var , add=True)
                
                action = 'keep' # we already changed, now back to business
                        
            #################################################################
            #                                                               #
            #    Before doing anything, the development bank must collect   #
            #      its  payments and guarantee contracts that need be       #
            #                                                               #
            #################################################################
            
            #first, we must update the risk dictionary
                
                
            for source in [0,1,2,3,4,5]:
                try:
                    RISKS.update({source : finding_FF(AGENTS.get(env.now), what, 'median', {'genre' : 'BB'}).get('value')})
                    #we attempt to get the median risk for each source
                except:
                    #if not possible (there are no banks financing that source or no banks at all) which translates to risky source
                    RISKS.update({source : 1})
            
            if env.now>0:
                                 
                for _ in CONTRACTS.get(env.now-1):
                    i = CONTRACTS.get(env.now-1).get(_)
                    if i.get('receiver') == 'DBB' and i.get('status') == 'payment':
                        """ the contract is adressed to the development bank and is a payment"""
                        receivable.update({
                            i.get('source') : receivable.get(i.get('source')) - i.get('value')
                                  })
                        wallet     += i.get('value')
                    elif i.get('receiver') == 'DBB' and i.get('guarantee') == True:
                        """ we are dealing with a guarantee"""
                        wallet     += i.get('value')
                        
                        if i.get('status') == 'default':
                            """ the company went into deafult and the guarantee is activated"""
                            j=i.copy()
                            j.update({'sender' : 'DBB',
                                     'receiver' : i.get('BB'),
                                     'status' : 'payment'})
                            guaranteed_contracts.append(j)
                        
                    if len(guaranteed_contracts) > 0:
                        for i in guaranteed_contracts:
                            if i.get('ammortisation') > env.now:
                                code=uuid.uuid4().int
                                j = i.copy()
                                CONTRACTS.get(env.now).update({code : j})
                                receivable.update({
                                    i.get('source') : receivable.get(i.get('source')) - i.get('value')
                                  })
                                wallet     += i.get('value')
                            else:
                                guaranteed_contracts.remove(i)
            
            #################################################################
            #                                                               #
            #           First, the DBB adds or changes its policy           #
            #                                                               #
            ################################################################# 
            
            policy_pool = []
            policy_pool.append(policy)
            policy_pool.append(policies) #with this we a temporary list with first the current policy and afterwars all the other policies
                
            if env.now >=2 :
                for entry in policy_pool:
                    instrument = entry.get('instrument')
                    source     = entry.get('source')
                    budget     = entry.get('budget') if 'budget' in entry else value * wallet
                    value      = disclosed_var if 'value' not in entry else entry.get('value')
                    
                    if instrument == 'finance' and len(CONTRACTS.get(env.now-1))>0:
                        financing=financing_FF(genre, target, name, wallet, receivable, value, financing_index)

                        wallet     = financing.get('wallet')
                        receivable = financing.get('receivables')

                    else:
                        """ We are dealing with guarentees """
                        financing = financing_FF(genre, target, name, wallet, receivable, value, financing_index, True)

                        wallet     = financing.get('wallet')
                        receivable = financing.get('receivables')

                    """ and now back to the actual variables for the current policy"""
                instrument  = Policies[0]['instrument']
                source      = Policies[0]['source']  
                value       = disclosed_var
            
            #################################################################
            #                                                               #
            #    Now, the EPM analyses what happened to the system due to   #
            #                       its intervention                        #
            #                                                               #
            #################################################################

            add_source = source_reporting_FF(name)
            
            for entry in dd_source['ranks']:
                dd_source['ranks'][entry] *= (1-discount)
                dd_source['ranks'][entry] += add_source[entry]
            
            #################################################################
            #                                                               #
            #         And then, the EPM will decide what to do next         #
            #                                                               #
            #################################################################

            if env.now>0:
                
                decision_var = max(0,min(1, public_deciding_FF(name)))
                disclosed_var = thresholding_FF(responsivity, disclosed_var, decision_var)

            #################################################################
            #                                                               #
            #    Before leaving, the agent must uptade the outside world    #
            #                                                               #
            #################################################################
    
            AGENTS.get(env.now).update({
                name : {
                    "NPV_THRESHOLD_DBB"    : NPV_THRESHOLD_DBB    ,
                    "guaranteed_contracts" : guaranteed_contracts ,
                    "genre"                : genre                ,
                    "subgenre"             : subgenre             ,
                    "name"                 : name                 ,
                    "wallet"               : wallet               ,
                    "dd_policy"            : dd_policy            ,
                    "dd_source"            : dd_source            ,
                    "decision_var"         : decision_var         ,
                    "disclosed_var"        : disclosed_var        ,
                    "action"               : action               ,
                    "dd_responsiveness"    : dd_responsiveness    ,
                    "dd_qual_vars"         : dd_qual_vars         ,
                    "dd_backwardness"      : dd_backwardness      ,
                    "dd_avg_time"          : dd_avg_time          ,
                    "dd_discount"          : dd_discount          ,
                    "dd_policy"            : dd_policy            ,
                    "policies"             : policies             ,
                    "dd_index"             : dd_index             ,
                    "index_per_source"     : index_per_source     ,
                    "dd_eta"               : dd_eta               ,
                    "dd_ambition"          : dd_ambition          ,
                    "dd_target"            : dd_target            ,
                    "dd_rationale"         : dd_rationale         ,
                    "financing_index"      : financing_index      ,
                    "Portfolio"            : Portfolio            ,
                    "receivable"           : receivable           ,
                    "accepted_sources"     : accepted_sources     ,
                    "car_ratio"            : car_ratio            ,
                    "policy"               : policy               ,
                    "source"               : source               ,
                    "responsiveness"       : responsiveness       ,
                    "backwardness"         : backwardness         ,
                    "avg_time"             : avg_time             ,
                    "discount"             : discount             ,
                    "index"                : index                ,
                    "eta_acc"              : eta_acc              ,
                    "ambition"             : ambition             ,
                    "rationale"            : rationale            ,
                    "value"                : value                
                }})

            yield env.timeout(1)

In [34]:
class BB(object):
    def __init__(self,env):
        self.env = env
        self.financing_index    = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
        self.Portfolio          = STARTING_PORTFOLIO
        self.receivable         = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
        self.accepted_sources   = ACCEPTED_SOURCES
        self.car_ratio          = 0
        self.name               = STARTING_NAME #name of the agent, is a string, normally something like BB_01
        self.genre              = 'BB' #genre, we do not use type, because type is a dedicated command of python, is also a string
        self.subgenre           = STARTING_TECHNOLOGY.get('source') #subgenre or source, we used to use subgenre a lot, now it's kind of a legacy. Is a number, 1 is wind, 2 is solar, 4 is biomass and 5 is hydrogen. 0 and 3 are not used because they are the fossil options
        self.wallet             = STARTING_WALLET #wallet or reserves, or savings, etc. How much the agent has? is a number
        self.profits            = 0 #profits of the agent, is a number
        self.dd_profits         = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0} #same as profits, but as dict. Makes accounting faster and simpler
        self.dd_source          = STARTING_DD_SOURCE #This, my ganzirosis, used to be the Tactics. It is the first of the ranked dictionaries. It goes a little sumthing like dis: dd = {'current' : 2, 'ranks' : {0: 3500, 1: 720, 2: 8000}}. With that we have the current decision for the variable or thing and on the ranks we have the score for 
        self.decision_var       = STARTING_DECISION_VAR #this is the value of the decision variable. Is a number between -1 and 1
        self.action             = "keep" #this is the action variable. It can be either 'keep', 'change' or 'add' 
        self.dd_responsiveness  = STARTING_RESPONSIVENESS #this is the responsiveness, follows the current ranked dictionary
        self.dd_qual_vars       = STARTING_QUAL_VARS #this tells the agent the qualitative variables in a form {0 : 'name of the zeroth variable', 1 : 'name of the first variable', 2 : 'name of the second variable'}
        self.dd_backwardness    = STARTING_BACKWARDNESS #also a ranked dictionary, this one tells the backwardness of agents
        self.dd_avg_time        = STARTING_DD_AVG_TIME #also a ranked dictionary, this one tells the average time for deciding if change is necessary
        self.dd_discount           = STARTING_DISCOUNT #discount factor. Is a ranked dictionary
        self.dd_strategies      = STARTING_STRATEGIES #initial strategy for the technology provider. Is a ranked dictionary
        
        self.action = env.process(self.run_BB(self.financing_index    , 
                                              self.Portfolio          , 
                                              self.receivable         , 
                                              self.accepted_sources   , 
                                              self.car_ratio          , 
                                              self.name               , 
                                              self.genre              , 
                                              self.subgenre           , 
                                              self.wallet             , 
                                              self.profits            , 
                                              self.dd_profits         , 
                                              self.dd_source          , 
                                              self.decision_var       , 
                                              self.action             , 
                                              self.dd_responsiveness  , 
                                              self.dd_qual_vars       , 
                                              self.dd_backwardness    , 
                                              self.dd_avg_time        , 
                                              self.dd_discount        , 
                                              self.dd_strategies      
                                             ))        
        
    def run_BB(self,
               financing_index    , 
               Portfolio          , 
               receivable         , 
               accepted_sources   , 
               car_ratio          , 
               name               , 
               genre              , 
               subgenre           , 
               wallet             , 
               profits            , 
               dd_profits         , 
               dd_source          , 
               decision_var       , 
               action             , 
               dd_responsiveness  , 
               dd_qual_vars       , 
               dd_backwardness    , 
               dd_avg_time        , 
               dd_discount        , 
               dd_strategies      ):
        
        global CONTRACTS
        global MIX
        global AGENTS
        global AGENTS_r
        global TECHNOLOGIC
        global r
        global BASEL
        global AMMORT
        global TACTIC_DISCOUNT
        global NPV_THRESHOLD
        global RISKS
        
        while True:
            
            #################################################################
            #                                                               #
            #     Before anything, we must the current values of each of    #
            #        the dictionaries that we use and other variables       #
            #                                                               #
            #################################################################
            
            list_of_strikeables = [dd_responsiveness, dd_qual_vars, dd_backwardness, dd_avg_time, dd_discount, dd_strategies]
            
            source         = dd_source.get(        'current')         
            responsiveness = dd_responsiveness.get('current') 
            qual_vars      = dd_qual_vars.get(     'current')      
            backwardness   = dd_backwardness.get(  'current')   
            avg_time       = dd_avg_time.get(      'current')
            discount       = dd_discount.get(      'current')
            strategy       = dd_strategies.get(    'current')
            discount       = dd_discount['current']
            index          = indexing_FF('TPM') if env.now>0 else dd_index['current']
            value          = decision_var
            profits        = 0 #in order to get the profits of this period alone
            
            #################################################################
            #                                                               #
            #                First, the bank collect profits                #
            #                                                               #
            #################################################################

            if env.now>0:
                for _ in CONTRACTS.get(env.now-1):
                    i = CONTRACTS.get(env.now-1).get(_)
                    if i.get('receiver') == name and i.get('status') == 'payment':
                        profits    += i.get('value')
                        wallet     += i.get('value')
                        receivable.update({
                            i.get('source') : receivable.get(i.get('source')) - i.get('value')
                                  })
                        dd_profits.update({
                            i.get('source') : dd_profits.get(i.get('source')) - i.get('value')
                                  })

            #################################################################
            #                                                               #
            #    Now, on to check if change is on and if there is a strike  #
            #                                                               #
            #################################################################
            
            if env.now > 0 and (action == 'add' or 'change'):
                striked = striking_FF(list_of_strikeables)
                
                for entry in range(0,len(list_of_strikeables)):
                    list_of_strikeables[entry] = striked[entry]
                    
                if source != dd_source.get('current'):
                    # we changed the source, so we have to update the accepted_sources dictionary
                    source_accepting(accepted_sources, source)
                    source = dd_source.get('current')
                action = 'keep' # we already changed, now back to business
                
                
            #################################################################
            #                                                               #
            #        Then, the bank decides which projects to accept        #
            #                                                               #
            #################################################################

            if env.now>0 and len(CONTRACTS.get(env.now-1))>0:
                financing=financing_FF(genre, accepted_sources, name, wallet, receivable, value, financing_index)

                wallet     = financing.get('wallet'     )
                receivable = financing.get('receivables')

            #################################################################
            #                                                               #
            #    Now, the BB analyses what happened to the system due to    #
            #                       its intervention                        #
            #                                                               #
            #################################################################

            add_source = source_reporting_FF(name)
            
            for entry in dd_source['ranks']:
                dd_source['ranks'][entry] *= (1-discount)
                dd_source['ranks'][entry] += add_source[entry]
            
            #################################################################
            #                                                               #
            #          And then, the BB will decide what to do next         #
            #                                                               #
            #################################################################

            if env.now>0:
                
                decision_var = max(0,min(1, private_deciding_FF(name)))

            #################################################################
            #                                                               #
            #    Before leaving, the agent must uptade the outside world    #
            #                                                               #
            #################################################################

            AGENTS.get(env.now).update({name : {
                "financing_index"   : financing_index    ,
                "Portfolio"         : Portfolio          ,
                "receivable"        : receivable         ,
                "accepted_sources"  : accepted_sources   ,
                "car_ratio"         : car_ratio          ,
                "name"              : name               ,
                "genre"             : genre              ,
                "subgenre"          : subgenre           ,
                "wallet"            : wallet             ,
                "profits"           : profits            ,
                "dd_profits"        : dd_profits         ,
                "dd_source"         : dd_source          ,
                "decision_var"      : decision_var       ,
                "action"            : action             ,
                "dd_responsiveness" : dd_responsiveness  ,
                "dd_qual_vars"      : dd_qual_vars       ,
                "dd_backwardness"   : dd_backwardness    ,
                "dd_avg_time"       : dd_avg_time        ,
                "dd_discount"       : dd_discount        ,
                "dd_strategies"     : dd_strategies      ,
                "source"            : source             ,
                "responsiveness"    : responsiveness     ,
                "qual_vars"         : qual_vars          ,
                "backwardness"      : backwardness       ,
                "avg_time"          : avg_time           ,
                "discount"          : discount           ,
                "strategy"          : strategy           ,
                "discount"          : discount           ,
                "index"             : index              ,
                "value"             : value              ,
                                               }})
            
            profits_dedicting_FF(name)
            post_evaluating_FF(decisions['strikes'], name)
        
            yield env.timeout(1)

In [18]:
class EP(object):
    def __init__(self,env):
        self.env = env
        self.genre                 = 'EP'
        self.subgenre              = STARTING_SUBGENRE
        self.name                  = STARTING_NAME
        self.wallet                = STARTING_WALLET
        self.profits               = 0
        self.EorM                  = STARTING_eORm
        self.capacity              = {0 : 0, 1: 0, 2: 0}  if self.EorM == 'E' else {3: 0, 4: 0, 5:0}
        self.Tactics               = STARTING_TACTICS
        self.SAV                   = STARTING_STRATEGY
        self.project_lifetime      = 12
        self.portfolio_of_plants   = {}
        self.portfolio_of_projects = {}
        self.periodicity           = 3
        self.MW_dict               = STARTING_MW_dict
        self.subgenre_price        = {0 : 0, 1: 0, 2: 0}  if self.EorM == 'E' else {3: 0, 4: 0, 5:0}
        self.capacity_to_add       = 1
        self.tolerance             = 12
        self.last_acquisition_period = 0
        
        
        
        self.action = env.process(self.run_EP(self.genre                 ,
                                              self.subgenre              ,
                                              self.name                  ,
                                              self.wallet                ,
                                              self.profits               ,
                                              self.capacity              ,
                                              self.EorM                  ,
                                              self.Tactics               ,
                                              self.SAV                   ,
                                              self.project_lifetime      ,
                                              self.portfolio_of_plants   ,
                                              self.portfolio_of_projects ,
                                              self.periodicity           ,
                                              self.subgenre_price        ,
                                              self.capacity_to_add       ,
                                              self.tolerance             ,
                                             self.last_acquisition_period,
                                             self.MW_dict))        
        
    def run_EP(self,
               genre                 ,
               subgenre              ,
               name                  ,
               wallet                ,
               profits               ,
               capacity              ,
               EorM                  ,
               Tactics               ,
               SAV                   ,
               project_lifetime      ,
               portfolio_of_plants   ,
               portfolio_of_projects ,
               periodicity           ,
               subgenre_price        ,
               capacity_to_add       ,
               tolerance             ,
              last_acquisition_period,
              MW_dict):
            global CONTRACTS
            global MIX
            global AGENTS
            global TECHNOLOGIC
            global r
            global DEMAND
            global KICKSTART_ADDITION
            
            while True:

                #############################################################
                #                                                           #
                #  First, the Energy producer collects its profits from the #
                #      previous period and pays the OPEX of its plants      #
                #                                                           #
                #############################################################

                if env.now>0 and len(MIX.get(env.now-1))>0:
                    """ The EP only collects profits if it produces energy and that can only happen if we are not in the start of the simulation, and if the MIX is not empty"""
                    """ Moreover, if the plant was activated, then the EP pays the OPEX of it """
                    for _ in MIX.get(env.now-1):
                        """ Then we check each plant in the mix """
                        i = MIX.get(env.now-1).get(_)
                        """ the _ is the code of the plant, whereas i is the dictionary of the plant itself """
                        if i.get('EP') == name and i.get('status') == 'contracted':
                            """ if the plant is mine and it is contracted, I'll collect profits """
                            wallet  += i.get('MWh') * i.get('price') - i.get( 'OPEX')
                            profits += i.get('MWh') * i.get('price') - i.get( 'OPEX')
                            
                            """ we also have to put the profit as a contract in the CONTRACTS dictionary in order for the policy makers, other EPs and the demmand to do some calculations """
                            code = uuid.uuid4().int
                            """ this is to get a unique and random number """
                            j = i.copy()
                            """ and we also have to create a copy of the i dictionary, if not things will update on that dictionary, and that's no good """
                            j.update({
                                'status'   : 'payment',
                                'sender'   : 'D',
                                'source'   : j.get('source'),
                                'receiver' : name,
                                'value'    : j.get('MWh') * j.get('price')
                            })
                            """ and now we update"""
                            CONTRACTS.get(env.now-1).update({code : j})

                #############################################################
                #                                                           #
                #  Now the EP goes through its portfolio_of_plants in order #
                #   to 1) pay the banks for the financing, 2) check if      #
                #     plants finished building, 3) retire old plants and    #
                #               4) insert plants into the mix               #
                #                                                           #
                #############################################################
                
                
                """ before anything, we have to make sure that the MW_dict of this period has the same MW as the last period """
                MW_dict.update({env.now + 1 : MW_dict.get(env.now)})
                if len(portfolio_of_plants)>0:
                    for _ in portfolio_of_plants:
                        i = portfolio_of_plants.get(_)
                        number = ((i.get('principal') / (1 + AMMORT)) + i.get('principal') * r)
                        
                        """  1) we pay for the ammortization of plants """
                        
                        if i.get('ammortisation') > env.now and (i.get( 'guarantee') != True or (i.get('guarantee') == True and wallet >= number)):                            
                            """ if the plant is not guaranteed or if it's guaranteed but the EP has enough money to cover it """
                            wallet  -=  number
                            
                            code  =  uuid.uuid4().int
                            CONTRACTS.get(env.now).update({
                                code : {
                                    'sender'   : name,
                                    'receiver' : i.get('BB'),
                                    'status'   : 'payment',
                                    'source'   : i.get('source'),
                                    'value'    : number
                                }
                            })   
                        elif i.get('ammortisation') > env.now and i.get('guarantee') == True and wallet < number:
                            """ if not, then the development bank pays for that monthly fee"""
                            j = i.copy()
                            j.update({
                                'receiver' : 'DBB',
                                'status'   : 'default'
                            })
                            code = uuid.uuid4().int
                            CONTRACTS.get(env.now).update({code : j})
                        
                        """ 2) now we retire old pants """
                        
                        if i.get('retirement') <= env.now:
                            i.update({'status' : 'retired'})
                            
                        """ 3) and we check if plants finished building """
                        
                        if i.get('completion') <=  env.now:
                            i.update({'status' : 'built'})
                            
                        """ 4) now we insert built plants into the mix  """
                            
                        if i.get('status') ==  'built':
                            j = i.copy()
                            MIX.get(env.now).update({_ : j})
                            
                #############################################################
                #                                                           #
                #      Now, the EP has to check if projects that were       #
                #     auction contracted or had guarantees got financed.    #
                #         If not, they need to be inserted into the         #
                #      portfolio_of_projects dictionary in order to be      #
                #       retired each period until the TOLERANCE limit       #
                #                                                           #
                #############################################################

                if env.now > 0 and len(CONTRACTS.get(env.now-1)) > 0:
                    for _ in CONTRACTS.get(env.now-1):
                        i = CONTRACTS.get(env.now-1).get(_) 
                        if i.get('receiver') == name and i.get('status') == 'financed':
                            """ the project was financed """
                            last_acquisition_period=env.now
                            j = i.copy() 
                            j.update({
                                'EP'         : name,
                                'BB'         : j.get('sender'),
                                'code'       : _,
                                'status'     : 'building',
                                'principal'  : j.get('CAPEX') * (1+r) ** j.get('building_time'),
                                'completion' : j.get('building_time') + 1 + env.now,
                                'retirement' : j.get('lifetime') + j.get('building_time') + 1 + env.now
                            })
                            
                            """ moreover, if it is a molecule project,
                            the price is pre-fixed """
                            if i.get('EorM') == 'M':
                                j.update({
                                    'price' : (i.get('OPEX') +(i.get('CAPEX') / i.get('lifetime')))/i.get('MWh') * (1 + SAV[2])
                                })
                            #j.pop('receiver')
                            #j.pop('sender')
                            
                            portfolio_of_plants.update({_ : j})
                            #print('portfolio_of_plants.update({_ : j})', _, j)
                            capacity.update({i.get('source') : capacity.get( i.get('source')) + i.get('capacity')})
                            MW_dict.update({env.now : MW_dict.get(env.now) + i.get('capacity')})
                            
                            """ if the financed project was in the pool of projects of the EP, we have to take it out """
                            if _ in portfolio_of_projects:
                                portfolio_of_projects.pop(_)

                                """ now we ready the contract that tells the TP that he got a new project"""
                            code = uuid.uuid4().int
                            CONTRACTS.get(env.now).update({
                                code : {'sender'   : name,
                                        'receiver' : j.get('TP')    ,
                                        'status'   : 'payment'      ,
                                        'source'   : j.get('source'),
                                        'value'    : j.get('CAPEX') ,
                                        'MWh'      : i.get('MWh')
                                       }
                            })
                        elif (i.get('guarantee') == True or i.get('auction_contracted') == True) and i.get('receiver') == name and i.get('status') == 'project' and 'failed_attempts' not in i:
                            """ if the project was not financed but it got a guarantee or whas a PPA, we have to prepare it to be be inserrted into the portfolio_of_projects dictionary """
                            j = i.copy()
                            j.update({'code' : _})
                            if 'limit' not in j:
                                """ if the key 'limit' is not in j, then we insert it, as well as the list failed_attempts, in which we put the name of the banks that rejected the project"""
                                j.update({'limit' : env.now + tolerance})
                                j.update({'failed_attempts' : [i.get('receiver')]})
                            portfolio_of_projects.update({_ : j})
                        elif _ in portfolio_of_projects and i.get('sender') == name and i.get('status') == 'rejected' and j.get('limit') > env.now and j.get('CAPEX') <= wallet:
                            """ the rejected project was in the portfolio_of_projects dicionary. So now we have to decide it will be prepared to be resent to another bank or if the EP will finance it through reinvestment"""
                            j = i.copy()
                            """ if no one accepts to finance and the EP has enough money, it can pay for the plant itself"""
                            wallet -= j.get('CAPEX')
                            portfolio_of_projects.pop(j.get('code'))
                            code = uuid.uuid4().int
                            last_acquisition_period=env.now
                            k = i.copy()
                            k.update({
                                'EP' : name,
                                'BB' : 'reinvestment',
                                'code': code,
                                'status' : 'building',
                                'principal' : 0,
                                'completion' : k.get('building_time') + 1 + env.now,
                                'retirement' : k.get('lifetime') + k.get('building_time') + 1 + env.now
                            })
                            """ moreover, if it is a molecule project, the price is pre-fixed """
                            if i.get('EorM') == 'M':
                                k.update({'price' : (i.get('OPEX') + (i.get('CAPEX') / i.get('lifetime'))) * SAV[2]})
                            portfolio_of_plants.update({code : k})  
                            capacity.update({k.get('source') : capacity.get(k.get('source')) + k.get('capacity')})
                            MW_dict.update({env.now : MW_dict.get(env.now) + k.get('capacity')})
                            
                            code = uuid.uuid4().int
                            CONTRACTS.get(env.now).update({
                                code :
                                    {'sender' : name,
                                     'receiver' : k.get('TP'),
                                     'status' : 'payment',
                                     'source' : k.get('source'),
                                     'value' : k.get('CAPEX'),
                                     'MWh' : k.get('MWh')
                                    }
                            })
                        elif _ in portfolio_of_projects and i.get('sender') == name and i.get('status') == 'rejected' and j.get('limit') > env.now and j.get('CAPEX') > wallet:
                            """ if not, then we take that project from the pool"""
                            portfolio_of_projects.pop(j)
                        elif _ in portfolio_of_projects and i.get('sender') == name and i.get('status') == 'rejected' and j.get('limit') < env.now:
                            portfolio_of_projects.get(i.get('code').update({
                                'failed_attempts' : j.get( 'failed_attempts' ).append(j.get('receiver'))}))
                
                #############################################################
                #                                                           #
                #   Then, the Energy producer decides how much to invest    #
                #                                                           #
                #############################################################

                source = SAV[0]
                action = SAV[1]
                value  = SAV[2]

                if action == 'change':
                    choice=min(np.random.poisson(1), 2)
                    source = sorted(list(Tactics.get(env.now-1).items()), key = lambda x: x[1], reverse=True)[choice][0]

                if (env.now % periodicity == 0 and env.now> 1 + periodicity) or env.now == 1:
                    """ first, we get how much would the energy producer like to insert"""
                    if env.now !=1:
                        capacity_t = 0
                        for _ in portfolio_of_plants:
                            plant = portfolio_of_plants.get(_)
                            if plant.get('status') == 'built':
                                capacity_t += plant.get('capacity')
                        demmand_t = DEMAND.get(env.now-1).get(EorM)
                        demmand_t_p = DEMAND.get(last_acquisition_period).get(EorM)
                        capacity_t_p = MW_dict.get(last_acquisition_period)

                        capacity_to_add = (capacity_t *(1  +  value)  *  demmand_t  /  demmand_t_p) - capacity_t_p
                        
                    else:
                        capacity_t = 0
                        capacity_to_add = KICKSTART_ADDITION.get(EorM)

                    """  and then we get which EP to try"""

                    TP = {'TP'    : 0,
                          'NPV'   : False,
                          'Lumps' : 0,
                          'CAPEX' : 0,
                          'OPEX'  : 0
                         }
                    for _ in TECHNOLOGIC.get(env.now-1):
                        i = TECHNOLOGIC.get(env.now-1).get(_)
                        if i.get('source') == source:
                            subgenre_price=weighting_FF(env.now-1, 'price', 'MWh', MIX)
                            Lumps = np.ceil(capacity_to_add / i.get('MW'))
                            price = subgenre_price.get(i.get('source'))
                            NPV = npv_generating_FF(r, i.get('lifetime'), Lumps, Lumps*i.get('MW'), i.get('building_time'), i.get('CAPEX'), i.get('OPEX'), price, i.get('CF'), AMMORT)
                            if NPV>TP.get('NPV') or TP.get('NPV') == False:
                                TP.update({
                                    'TP'           : _,
                                    'NPV'          : NPV,
                                    'Lumps'        : Lumps,
                                    'CAPEX'        : i.get('CAPEX') * Lumps,
                                    'OPEX'         : i.get('OPEX')  * Lumps,
                                    'source_of_TP' : i.get('source')
                                })

                    if source in AUCTION_WANTED_SOURCES:
                        receiver  =  'EPM'
                    else:
                        """ if the source is not currently in an auction, the EP sends it directly to a bank"""
                        """ now we select which bank to try """
                        BB_=[]
                        for _ in AGENTS.get(env.now-1):
                            """ we must check at this period, because the EP goes after the bank """
                            agent=AGENTS.get(env.now-1).get(_)
                            if agent.get('genre') == 'BB' or agent.get('genre') == 'DBB':
                                BB_.append([agent.get('name'), agent.get('financing_index').get(source)])
                        number = np.random.poisson(1)
                        print(BB_)
                        number = number if number < len(BB_) else len(BB_)-1
                        BB = sorted(BB_, key = lambda x: x[1], reverse = True)[number][0]
                        # first we sort the list of agents. With the .values() we have both keys and values. The [number] selects the random pick of which source to get. The .get('name') selects the name of the bank
                        receiver=BB

                    """ and now we send that project to the bank """
                    if capacity_to_add>0:
                        #OPEX and CAPEX are in relation to one lump, so in the project we have to change them to account for the whole project
                        project = TECHNOLOGIC.get(env.now-1).get( TP.get('TP') ).copy()
                        # we have to use .copy() here to avoid changing the TECHNOLOGIC dictionary entry
                        project.update({
                            'sender'    : name,
                            'receiver'  : receiver,
                            'TP'        : TP.get('TP'),
                            'Lumps'     : TP.get('Lumps'),
                            'old_CAPEX' : TECHNOLOGIC.get(env.now-1).get( TP.get('TP')).get('CAPEX'),
                            'old_OPEX'  : TECHNOLOGIC.get(env.now-1).get( TP.get('TP')).get('OPEX'),
                            'CAPEX'     : TP.get('CAPEX'),
                            'OPEX'      : TP.get('OPEX'),
                            'status'    : 'project',
                            'capacity'  : Lumps * project.get('MW'),
                            'MWh'       : Lumps * project.get('MW') * 24 * 30 * project.get('CF')})
                        if TP.get('source_of_TP') in AUCTION_WANTED_SOURCES:
                            project.update(
                                {'status' : 'bidded',
                                'receiver' : 'EPM'})
                        else:
                            project.update({'status' : 'project'})
                        code=uuid.uuid4().int
                        CONTRACTS.get(env.now).update({code : project})
                                                      
                for _ in portfolio_of_projects:
                    """ now we have to resend the "projects in the portfolio_of_projects dictionary """
                    i = portfolio_of_projects.get(_).copy()
                    number = np.random.poisson(1)
                    BB_list = []
                    for j in sorted(list(AGENTS.get(env.now-1).values()), key = lambda x: x.get('financing_index').get(i.get('source')), reverse = True):
                        BB_list.append(j.get('name'))
                    for bank in i.get('failed_attempts'):
                        BB_list.remove(bank)
                    BB = BB_list[number] if number < len(BB_list) else BB_list[-1]
                    project = i.copy()
                    project.update({'sender' : name,
                                   'receiver' : BB,
                                   'status' : 'project'})
                    CONTRACTS.get(env.now).update({
                        project.get('code') : {project}
                    })      
                
                #############################################################
                #                                                           #
                #   Afterwards, the Energy provider evaluates the context   #
                #                                                           #
                #############################################################

                if env.now>0:
                    vote=voting(genre, env.now, name, EorM)
                    for Source in list(Tactics.get(0).keys()):
                        Tactics.get(env.now).update({Source : Tactics.get(env.now-1).get(Source) * TACTIC_DISCOUNT + vote.get(Source)})
                
                #############################################################
                #                                                           #
                #   And finally, the Energy producer decides what to do in  #
                #                      the next period                      #
                #                                                           #
                #############################################################

                if env.now>0:
                    decision = evaluation_criteria(genre, subgenre, Tactics, source)     
                    value_new = min(100, max(0, value + (decision.get('value') / 100)))
                    SAV=[SAV[0], decision.get('action'), value_new]

                #############################################################
                #                                                           #
                #  Before leaving, the agent must update the outside world  #
                #                                                           #
                #############################################################

                AGENTS.get(env.now).update({
                    name :
                    {"genre"                 : genre                 ,
                     "subgenre"              : subgenre              ,
                     "name"                  : name                  ,
                     "wallet"                : wallet                ,
                     "profits"               : profits               ,
                     "capacity"              : capacity              ,
                     "EorM"                  : EorM                  ,
                     "Tactics"               : Tactics.get(env.now)  ,
                     "SAV"                   : SAV                   ,
                     "project_lifetime"      : project_lifetime      ,
                     "portfolio_of_plants"   : portfolio_of_plants   ,
                     "portfolio_of_projects" : portfolio_of_projects ,
                     "periodicity"           : periodicity           ,
                     "subgenre_price"        : subgenre_price        ,
                     "capacity_to_add"       : capacity_to_add       ,
                     "tolerance"             : tolerance             }
                })
                yield env.timeout(1)

In [19]:
class Demand(object):
    def __init__(self,env):
        self.env              = env
        self.genre            = 'D'
        self.name             = 'D'
        self.initial_demand   = INITIAL_DEMAND
        self.specificities    = DEMAND_SPECIFICITIES
        self.action = env.process(self.run(self.genre         ,        
                                           self.name          ,         
                                           self.initial_demand,
                                           self.specificities    ))        
        
    def run(self,
           genre ,        
           name,  
           initial_demand,
           specificities):
        
        global CONTRACTS
        global MIX
        global AGENTS
        global TECHNOLOGIC
        global r
        global DEMAND
        
        while True:
            
            from_time_to_agents_FF(AGENTS)
            from_time_to_agents_FF(TECHNOLOGIC)
            
            if env.now % 10 == 0:
                print('time', env.now)
                
            when            = specificities.get('when')
            increase        = specificities.get('increase')
            green_awareness = specificities.get('green_awareness')
            
            if env.now == 0:
                DEMAND = {env.now : 
                          {'E' : initial_demand.get('E'),
                           'M' : initial_demand.get('M')}
                         }
            else:
                DEMAND.update({env.now : {'E' : DEMAND.get(env.now-1).get('E'),
                                     'M' : DEMAND.get(env.now-1).get('M')}})

            if env.now % when == 0 and env.now!=0:

                """ first, we get how much green is E or M"""
                greeness = {'E' : 0,'M' : 0}
                total = 0
                for month in range(env.now-when, env.now-1):
                    if len(MIX.get(month))>1:
                        for _ in MIX.get(month):
                            i=MIX.get(month).get(_)
                            if i.get('status') == 'contracted':
                                if i.get('green') == True:
                                    greeness.update({i.get('EorM') : greeness.get(i.get('EorM')) + i.get('MWh')})
                                total += i.get('MWh')
                            
                green = {'E' : greeness.get('E') / total, 'M' : greeness.get('M') / total}


                expected_increase = (DEMAND.get(env.now-1).get('E') + DEMAND.get(env.now-1).get('M')) * increase
                pendulum_demand = specificities.get('EorM') * expected_increase
                prices = weighting_FF(env.now-1, 'price', 'MWh', MIX, demand=True)
                a = green_awareness
                E_pendulum = a * (prices.get('E') / sum(list(prices.values()))) + (1 - a) * green.get('E')
                E_pendulum *= pendulum_demand
                E_increase = DEMAND.get(env.now-1).get('E') * specificities.get('increase') + E_pendulum
                M_pendulum = a * (prices.get('M')/sum(list(prices.values()))) + (1 - a) * green.get('M')
                M_pendulum *= pendulum_demand
                M_increase = DEMAND.get(env.now-1).get('M') * specificities.get('increase') + M_pendulum

                DEMAND.update({env.now : 
                               {'E' : (24 * 30) * DEMAND.get(env.now-1).get('E') + E_increase,
                                'M' : (24 * 30) * DEMAND.get(env.now-1).get('M') + M_increase}})
        
            yield env.timeout(1)

In [20]:
### We now automate the creation of agents

def Create(genre, traits):
    
    #PUBLIC AGENTS
    global SIM_TIME
    global STARTING_PPA_EXPIRATION
    global STARTING_PPA_LIMIT
    global STARTING_COUNTDOWN
    global STARTING_WALLET
    global STARTING_META_STRATEGY
    global STARTING_RIVAT
    global STARTING_POLICIES
    global STARTING_RANKS
    global STARTING_RANKS
    global STARTING_TACTICS
    global STARTING_NPV_THRESHOLD_DBB
    global STARTING_FINANCING_INDEX
    
    #PRIVATE AGENTS
    global STARTING_TECHNOLOGY
    global STARTING_NAME
    global STARTING_WALLET
    global STARTING_CAPACITY
    global STARTING_TACTICS
    global STARTING_STRATEGY
    global STARTING_RnD_THRESHOLD
    global STARTING_CAPACITY_THRESHOLD
    global STARTING_SUBGENRE
    global STARTING_eORm
    global STARTING_MW_dict
    global INITIAL_DEMAND
    global DEMAND_SPECIFICITIES
    global STARTING_RATIONALE
    global STARTING_PORTFOLIO
    global ACCEPTED_SOURCES
    global NUMBER_OF_TP_DICT
    global NPV_THRESHOLD_DBB
    global EP_NAME_LIST
    global TP_NAME_LIST
    global BB_NAME_LIST
    
    if env.now > 0:
        traits = AGENTS.get(env.now).get(traits)
    
    STARTING_WALLET = traits.get('wallet') # everyone has a wallet
    
    if genre in ('EPM', 'TPM', 'DBB'):
        """ common things to all policy makers """
        policy_maker_tactics={
            'innovation' : {1    : 0, 2    : 0,
                            4    : 0, 5    : 0,
                            12   : 0,
                            45   : 0,
                            1245 : 0},
            'capacity'   : {1    : 0, 2    : 0,
                            4    : 0, 5    : 0,
                            12   : 0,
                            45   : 0,
                            1245 : 0},
            'expansion'  : {1    : 0, 2    : 0,
                            4    : 0, 5    : 0,
                            12   : 0,
                            45   : 0,
                            1245 : 0}} 

        STARTING_META_STRATEGY= {0 :
                                 {0 : traits.get('meta_strategy').get( 'first_to_change'),
                                  'strikes' : 0,
                                  'max' : traits.get('meta_strategy').get( 'max_strikes_for_the_first_option')}, 1  :
                                 {1 : traits.get('meta_strategy').get( 'second_to_change'),
                                  'strikes' : 0,
                                  'max' : traits.get('meta_strategy').get( 'max_strikes_for_the_second_option')}, 2  :
                                 {2 : traits.get('meta_strategy').get( 'third_to_change'),
                                  'strikes' : 0,
                                  'max' : traits.get('meta_strategy').get( 'max_strikes_for_the_third_option')}}
        STARTING_RIVAT = [traits.get('RIVAT').get('Rationale'),
                         traits.get('RIVAT').get('Instrument'),
                         traits.get('RIVAT').get('Value'),
                         traits.get('RIVAT').get('Action'),
                         traits.get('RIVAT').get('Target')]
        STARTING_POLICIES=[]
        STARTING_RANKS= {'R' :
                         {'innovation' : traits.get('ranks').get('rationale' ).get('innovation'),
                          'expansion'  : traits.get('ranks').get('rationale' ).get('expansion'),
                          'capacity'   : traits.get('ranks').get('rationale' ).get('capacity')},
                        'I' : {}}
        for instrument_in_quotes in traits.get('ranks').get('instrument'):
            instrument = traits.get('ranks').get('instrument').get( instrument_in_quotes)
            STARTING_RANKS.get('I').update({
                instrument_in_quotes : instrument
            })
        STARTING_TACTICS={}
        for i in range(SIM_TIME):
            STARTING_TACTICS.update({i : policy_maker_tactics})
        if env.now > 0:
            STARTING_TACTICS = traits.get('Tactics')
            
        if genre == 'TPM':
            
            #################################################################
            #                                                               #
            #                    technology policy maker                    #
            #                                                               #
            #################################################################
            technology_policy_maker = TPM(env)
            
        elif genre == 'EPM':
            
            #################################################################
            #                                                               #
            #                      energy policy maker                      #
            #                                                               #
            #################################################################
            STARTING_PPA_EXPIRATION = traits.get('PPA_expiration') #for how many months do PPA contracts stand?
            STARTING_PPA_LIMIT = traits.get('PPA_limit') #how many months do companies have to build auction
            STARTING_COUNTDOWN = traits.get('auction_countdown') # how many months are spent between annoucing and deciding an auction?
            energy_policy_maker = EPM(env)
            
        else:
            
            #################################################################
            #                                                               #
            #                       Development Bank                        #
            #                                                               #
            #################################################################
            STARTING_NPV_THRESHOLD_DBB = traits.get( 'NPV_threshold_for_the_DBB')
            NPV_THRESHOLD_DBB = STARTING_NPV_THRESHOLD_DBB #it must be global for the finance function
            STARTING_FINANCING_INDEX   = {}
            for Source in traits.get('financing_index'):
                financing_index = traits.get('financing_index').get(Source)
                STARTING_FINANCING_INDEX.update({Source : financing_index})
            Development_Bank = DBB(env)
    else:
        """ so we are dealing with private agents """
        if genre == 'BB':
            
            #################################################################
            #                                                               #
            #                         Private Bank                          #
            #                                                               #
            #################################################################
            
            STARTING_SUBGENRE  = traits.get('subgenre')
            STARTING_NAME      = traits.get('name')
            BB_NAME_LIST.append(STARTING_NAME)
            STARTING_STRATEGY  = traits.get('strategy')
            STARTING_PORTFOLIO = traits.get('portfolio')
            ACCEPTED_SOURCES   = traits.get('accepted_sources')
            STARTING_RATIONALE = traits.get('rationale')
            
            STARTING_TACTICS = {}
            STARTING_TACTICS_LIST = {0:0,1:0,2:0,3:0,4:0,5:0}
            for j in range(SIM_TIME):
                for source in list(STARTING_TACTICS_LIST.keys()):
                    STARTING_TACTICS.update({j : STARTING_TACTICS_LIST})
            
            if env.now > 0:
                STARTING_TACTICS = traits.get('Tactics')
            
            STARTING_NAME=BB(env)
            
        elif genre == 'EP': 
            
            #################################################################
            #                                                               #
            #                       Energy Producer                         #
            #                                                               #
            #################################################################
            
            STARTING_STRATEGY = traits.get('strategy')
            STARTING_eORm = 'E' if STARTING_STRATEGY[0] in [0,1,2] else 'M'
            STARTING_SUBGENRE = STARTING_eORm
            STARTING_NAME = traits.get('name')
            EP_NAME_LIST.append(STARTING_NAME)
            STARTING_PERIODICITY = traits.get('periodicity')
            STARTING_TOLERANCE = traits.get('tolerance')
            STARTING_PORTFOLIO = []
            STARTING_TACTICS = {}
            STARTING_TACTICS_LIST={0:0,1:0,2:0} if STARTING_STRATEGY[0] in [0,1,2] else {3:0,4:0,5:0}
            STARTING_MW_dict={}
            for t in range(SIM_TIME):
                for source in list(STARTING_TACTICS_LIST.keys()):
                    STARTING_TACTICS.update({t : STARTING_TACTICS_LIST})
                STARTING_MW_dict.update({t : 0})
            
            if env.now > 0:
                STARTING_TACTICS = traits.get('Tactics')
            
            Energy_Producer = EP(env)
            
        else:
                        
            #################################################################
            #                                                               #
            #                     Technology Provider                       #
            #                                                               #
            #################################################################
            
            STARTING_NAME = traits.get('name')
            TP_NAME_LIST.append(STARTING_NAME)
            
            Tech_specs_dict = {1 : {'subgenre'    : 1,
                                    "transport"   : False,
                                    "source_name" : 'wind',
                                    'EorM'        : 'E'
                                   },
                               2 :  {'subgenre'    : 2,
                                     "transport"   : True,
                                     "source_name" : 'solar',
                                     'EorM'        : 'E'
                                    },
                               4 :  {'subgenre'    : 4,
                                     "transport"   : False,
                                     "source_name" : 'biogas',
                                     'EorM'        : 'M'
                                    },
                               5 :  {'subgenre'    : 5,
                                     "transport"   : False,
                                     "source_name" : 'hydrogen',
                                     'EorM'        : 'M'
                     }}
            traits.get('technology').update({
                'last_radical_innovation'  : 0   ,
                'last_marginal_innovation' : 0   ,
                'green'                    : True}) 
            traits.get('technology').update(
                Tech_specs_dict.get(traits.get('technology').get('source'))
            )
            
            STARTING_TECHNOLOGY = traits.get('technology')
            STARTING_TECHNOLOGY.update({'name' : STARTING_NAME})
            
            STARTING_SUBGENRE = traits.get('technology').get('subgenre')
            NUMBER_OF_TP_DICT.update({
                STARTING_SUBGENRE : NUMBER_OF_TP_DICT.get(STARTING_SUBGENRE) + 1
            })
            STARTING_CAPACITY = traits.get('wallet')
            EorM = traits.get('technology').get('EorM')
            STARTING_STRATEGY = traits.get('strategy')
            if EorM == 'E':
                tactical = (0,1,2)
            else:
                tactical = (3,4,5)
            
            STARTING_TACTICS = {}
            for t in range(SIM_TIME):
                STARTING_TACTICS.update({
                    t : {tactical[0] : 0, tactical[1] : 0, tactical[2] : 0}
                })
            STARTING_RnD_THRESHOLD = traits.get('RnD_threshold')
            STARTING_CAPACITY_THRESHOLD =  traits.get('capacity_threshold')
            
            if env.now > 0:
                STARTING_TACTICS = traits.get('Tactics')
            
            STARTING_NAME = TP(env)        
            
    return print('genre :', genre, ', traits:', traits, '. \n') 

In [22]:
####################################################
# First we start the environment and global things #
####################################################
####################################################

env = simpy.Environment(initial_time=0) #change initial time to allow replication
random.seed(1)
SIM_TIME=30*12 #how many months will the simulation run?
EP_NAME_LIST = []
TP_NAME_LIST = []
BB_NAME_LIST = [] #Initializing the llist with the names of the agents 
CONTRACTS={} #initializing the dictionary of contracts
AGENTS={} #initializing the dictionary of agents
MIX={} #initializing the dictionary of infrastructure
TECHNOLOGIC={} #initializing the dictionary of technologies
rev_dict = {}
r=0.001 #monthly real tax rate (already including inflation)
POLICY_EXPIRATION_DATE=12*10 #after how many months are the policies discarted?
TACTIC_DISCOUNT=0.99 #how much of past votes are carried out to the next period (1 is all, zero is none)
rNd_INCREASE = 0.5 #how much more difficult is to innovate after one innovation?
M_CONTRACT_LIMIT=2*12 #for how many months are molecule contracts contracted?
AUCTION_WANTED_SOURCES=[] #initializing the list of auctions currently on auction
AMMORT = 20*12 #how many months do companies have to pay for their projects?
NPV_THRESHOLD=0 #how much NPV should a project have to be accepted by a private bank?
INSTRUMENT_TO_SOURCE_DICT={1 : [1], 2 : [2], 12 : [1,2], 4 : [4], 5 : [5], 45 : [4,5], 1245 : [1,2,4,5]}
BASEL=0.105 #how much car_ratio can the banks have
MARGIN= .1 # how much margin should energy producers get for electricity projects?
INITIAL_DEMAND = {'E' : 100, 'M' : 80} # demmand in MW
KICKSTART_ADDITION = {'E' : INITIAL_DEMAND.get('E')/5,
                      'M' : INITIAL_DEMAND.get('M')} # how much MW to start adding to the system at t=1?
STARTING_PRICE = 25 #if there is no capacity yet, what should be the price for the sources?
RADICAL_THRESHOLD = 2 #how large should the a (the result of the probability distribution) be for that to be considered radical nnovation?
RISKS = {0: 0, 1:0, 2:0, 3:0, 4:0, 5:0} # risk dictionary for sources
NAMES = {'EP' : [], 'TP' : [], 'BB' : []}


##########################################
# Now, we start the Technology producers #
##########################################
##########################################

THERMAL = {"CAPEX"         : 20000000,
           'OPEX'          : 30000   ,
           "MW"            : 20      ,
           'CF'            : .7      ,
           "lifetime"      : 20 * 12 ,
           'building_time' : 12      }

GAS    = {"CAPEX"        : 20000000,
          'OPEX'         : 30000   ,
          "CF"           : .9      ,
          "MW"           : 5      ,
          "lifetime"     : 20 * 12 ,
          'building_time': 12      }

for j in range(SIM_TIME):
    CONTRACTS.update({j : {}})
    AGENTS.update({j : {}})
    MIX.update({j : {}})
    rev_dict({j : 0})
    TECHNOLOGIC.update({j : {'TP_thermal' : {'subgenre' : 0,
                "EorM" : 'E',
                'name' : 'TP_thermal',
                "green": False,
                "source": 0,
                "source_name": 'thermal',                             
                "CAPEX" : THERMAL.get("CAPEX"),
                'OPEX': THERMAL.get('OPEX'),
                "dispatchable": True,
                "transport": False,
                "CF": THERMAL.get("CF"),
                "MW": THERMAL.get("MW"),
                "lifetime": THERMAL.get("lifetime"),
                'building_time': THERMAL.get('building_time'),
                'last_radical_innovation' : 0,
                'last_marginal_innovation' : 0},
                             'TP_gas' : {'subgenre' : 3,
                "EorM" : 'M',
                "green": False,
                "source_name": 'gas',
                'name' : 'TP_gas',
                "source": 3,                         
                "CAPEX" :GAS.get("CAPEX"),
                'OPEX': GAS.get('OPEX'),
                "dispatchable": True,
                "transport": False,
                "CF": GAS.get("CF"),
                "MW": GAS.get("MW"),
                "lifetime": GAS.get("lifetime"),
                'building_time': GAS.get('building_time'),
                'last_radical_innovation' : 0,
                'last_marginal_innovation' : 0}}})
    
flagSHIPS = [4,4,4,4] # we want four technology providers for each source

Technologies_list=[{
    "source": 1,
    "CAPEX" : 20000000,
    'OPEX': 30000,
    "dispatchable": False,
    "CF": .29,
    "MW": 10,
    "lifetime": 20*12,
    'building_time': 12
},{
    "source": 2,
    "CAPEX" : 100000,
    'OPEX': 200,
    "dispatchable": False,
    "CF": .1,
    "MW": 1,
    "lifetime": 20*12,
    'building_time': 6
},{
    "source": 4,
    "CAPEX" : 20000000,
    'OPEX': 30000,
    "dispatchable": False,
    "CF": .74,
    "MW": 2,
    "lifetime": 20*12,
    'building_time': 12
},{
    "source": 5,
    "CAPEX" : 20000000,
    'OPEX': 30000,
    "dispatchable": False,
    "CF": .75,
    "MW": 50,
    "lifetime": 20*12,
    'building_time': 12
}]

TP_DICT = {'Strategies' : [
    ['capacity','keep',.25],
    ['capacity','keep',.25],
    ['capacity','keep',.25],
    ['capacity','keep',.25]
],
           'Capacities' : [5000000, 10000000, 20000000, 15000000] ,
           'Technologies' : Technologies_list                     ,
           'Wallets' : [10000000, 10000000, 10000000, 10000000]   ,
           'RnD_thresholds' : [5000000, 5000000, 5000000, 5000000],
           'Capacity_thresholds' : [3,3,3,3]
            }

NUMBER_OF_TP_DICT={0 : 0, 1 : 0, 2: 0,
                   3 : 0, 4: 0, 5: 0,
                   12 : 0, 45: 0, 1245 : 0}

for i in range(len(flagSHIPS)):
    for j in range(flagSHIPS[i]):
        traits = {
            'wallet' : TP_DICT.get('Wallets')[i],
            'name' : 'TP_'+str(i)+str(j),
            'subgenre' : TP_DICT.get('Technologies')[i].get('subgenre'),
            'strategy' : TP_DICT.get('Strategies')[i],
            'capacity' : TP_DICT.get('Capacities')[i],
            'technology' : TP_DICT.get('Technologies')[i],
            'RnD_threshold' : TP_DICT.get('RnD_thresholds')[i],
            'capacity_threshold' : TP_DICT.get('Capacity_thresholds')[i]
        }
        NAMES.get('TP').append(traits.get('name'))
        Create('TP', traits)

NUMBER_OF_TP_DICT.update({
            12 : NUMBER_OF_TP_DICT.get(1) + NUMBER_OF_TP_DICT.get(2)
})
NUMBER_OF_TP_DICT.update({
            45 : NUMBER_OF_TP_DICT.get(4) + NUMBER_OF_TP_DICT.get(5)
})
NUMBER_OF_TP_DICT.update({
            1245 : NUMBER_OF_TP_DICT.get(12) + NUMBER_OF_TP_DICT.get(45)
})

NUMBER_OF_TP=sum(flagSHIPS)+2

##################################
# And we start the private Banks #
##################################
##################################

#each entry is a different type of firm and the number is how many copies it will have
flagSHIPS = [3, 1] # we want 3 banks that only accept fossils and one that also accepts wind
BB_DICT = {'Strategies' : [[1,'keep',.1],[2,'keep',.25]],
          'Accepted_sources' : [
    {0: True, 1: False, 2: False, 3: True, 4: False, 5: False},
    {0: True, 1: True, 2: False, 3: True, 4: False, 5: False}],
          'Wallets' : [10000000, 10000000],
          'Portfolios' : [[],[]],
          'Rationales' : ['project_finance', 'project_finance']}

for i in range(len(flagSHIPS)):
    for j in range(flagSHIPS[i]):
        traits = {
            'wallet' : BB_DICT.get('Wallets')[i],
            'name' : 'BB_'+str(i)+str(j),
            'subgenre' : 'BB',
            'strategy' : BB_DICT.get('Strategies')[i],
            'portfolio' : BB_DICT.get('Portfolios')[i],
            'accepted_sources' : BB_DICT.get('Accepted_sources')[i],
            'rationale' : BB_DICT.get('Rationales')[i]            
        }
        NAMES.get('BB').append(traits.get('name'))
        Create('BB', traits)
NUMBER_OF_BB=sum(flagSHIPS)
    
##########################################
# And then we start the energy producers #
##########################################
##########################################

flagSHIPS = [9, 1, 9, 1] # we want 9 electricity producers that use fossil and one that uses wind, and 9 molecule producers that use fossil and one that uses biogas

EP_Dict = {'Strategies' : [
    [0,'keep',.1],
    [1,'keep',.25],
    [3,'keep',.1],
    [4,'keep',.25]
],
           'Wallets' : [10000000, 10000000, 10000000, 10000000],
           'Periodicities' : [3,3,3,3],
           'Tolerances' : [12,12,12,12]
            }

for i in range(len(flagSHIPS)):
    for j in range(flagSHIPS[i]):
        traits = {
            'wallet' : EP_Dict.get('Wallets')[i],
            'strategy' : EP_Dict.get('Strategies')[i],
            'name' : 'EP_'+str(i)+str(j),
            'periodicity' : EP_Dict.get('Periodicities')[i],
            'tolerance' : EP_Dict.get('Tolerances')[i]
        }
        NAMES.get('EP').append(traits.get('name'))
        Create('EP', traits)
NUMBER_OF_EP=sum(flagSHIPS)
    
#######################
# the demmand #
#######################
#######################

#INITIAL_DEMAND = {'E' : 3000*10**9, 'M' : 800*10**9} It was put at the beggining
DEMAND_SPECIFICITIES = {'when' : 60, 'green_awareness' : 0.1, 'EorM' : 0.5, 'increase' : 0.75}
DD=Demand(env)   
    
#############################################
# Now, we start the technology policy maker #
#############################################
#############################################

traits = {'wallet' : 4.5*10**9,
          'meta_strategy' : {
              'first_to_change'  : 'T',
              'max_strikes_for_the_first_option'  : 5,
              'second_to_change' : 'I',
              'max_strikes_for_the_second_option' : 3,
              'third_to_change'  : 'R',
              'max_strikes_for_the_third_option'  : 2},
          'RIVAT' : {'Rationale'  : 'innovation',
                     'Instrument' : 'supply',
                     'Value'      : 0.5,
                     'Action'     : 'keep',
                     'Target'     : 1245},
          'ranks' : {'rationale' : {'innovation' : 1,
                                   'capacity'    : 0,
                                   'expansion'   : 2},
                    'instrument' : {'supply' : 0}}
 }

Create('TPM', traits)

#########################################
# Now, we start the energy policy maker #
#########################################
#########################################

traits = {'wallet' : 4.5*10**9,
          'meta_strategy' : {
              'first_to_change'  : 'T',
              'max_strikes_for_the_first_option'  : 5,
              'second_to_change' : 'I',
              'max_strikes_for_the_second_option' : 3,
              'third_to_change'  : 'R',
              'max_strikes_for_the_third_option'  : 2},
          'RIVAT' : {'Rationale'  : 'innovation',
                     'Instrument' : 'carbon_tax',
                     'Value'      : 0.5,
                     'Action'     : 'keep',
                     'Target'     : 1245},
          'ranks' : {'rationale' : {'innovation' : 1,
                                   'capacity'    : 0,
                                   'expansion'   : 2},
                    'instrument' : {'carbon_tax' : 0,
                                   'FiT'         : 1,
                                   'auction'     : 2}},
          'PPA_expiration'    : 25*12,
          'PPA_limit'         : 5*12,
          'auction_countdown' : 6,
 }

Create('EPM', traits)

#######################################
# Now, we start the  Development Bank #
#######################################
#######################################

traits = {'wallet' : 4.5*10**9,
          'meta_strategy' : {
              'first_to_change'  : 'T',
              'max_strikes_for_the_first_option'  : 5,
              'second_to_change' : 'I',
              'max_strikes_for_the_second_option' : 3,
              'third_to_change'  : 'R',
              'max_strikes_for_the_third_option'  : 2},
          'RIVAT' : {'Rationale'  : 'capacity',
                     'Instrument' : 'guarantee',
                     'Value'      : 0.5,
                     'Action'     : 'keep',
                     'Target'     : 1245},
          'ranks' : {'rationale' : {'innovation' : 1,
                                   'capacity'    : 0,
                                   'expansion'   : 2},
                    'instrument' : {'finance'   : 0,
                                    'guarantee' : 1}},
          'NPV_threshold_for_the_DBB' : 0,
          'financing_index' : {
              0 : 0,
              1 : 0,
              2 : 0,
              3 : 0,
              4 : 0,
              5 : 0}
 }

Create('DBB', traits)

###################
# And now, we run #
###################
###################
t=40
#t=SIM_TIME
env.run(until=t)

genre : TP , traits: {'wallet': 10000000, 'name': 'TP_00', 'subgenre': None, 'strategy': ['capacity', 'keep', 0.25], 'capacity': 5000000, 'technology': {'source': 1, 'CAPEX': 20000000, 'OPEX': 30000, 'dispatchable': False, 'CF': 0.29, 'MW': 10, 'lifetime': 240, 'building_time': 12, 'last_radical_innovation': 0, 'last_marginal_innovation': 0, 'green': True, 'subgenre': 1, 'transport': False, 'source_name': 'wind', 'EorM': 'E', 'name': 'TP_00'}, 'RnD_threshold': 5000000, 'capacity_threshold': 3} . 

genre : TP , traits: {'wallet': 10000000, 'name': 'TP_01', 'subgenre': 1, 'strategy': ['capacity', 'keep', 0.25], 'capacity': 5000000, 'technology': {'source': 1, 'CAPEX': 20000000, 'OPEX': 30000, 'dispatchable': False, 'CF': 0.29, 'MW': 10, 'lifetime': 240, 'building_time': 12, 'last_radical_innovation': 0, 'last_marginal_innovation': 0, 'green': True, 'subgenre': 1, 'transport': False, 'source_name': 'wind', 'EorM': 'E', 'name': 'TP_01'}, 'RnD_threshold': 5000000, 'capacity_threshold': 3} .

In [23]:
"""
This is the basis for reworking the simulation with certain changes
"""

t=10

env = simpy.Environment(initial_time=10)
Create('DBB', 'DBB')
Create('EPM', 'EPM')
Create('TPM', 'TPM')

for genre in list(NAMES.keys()):
    for name in NAMES.get(genre):
        Create(genre, name)

env.run(until=2*t)

worked


AttributeError: 'list' object has no attribute 'get'